In [30]:
import soundfile as sf
from pydub import AudioSegment
import librosa
import numpy as np 
import os
import fnmatch
import numpy as np
import librosa
import torch # import PyTorch
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [48]:
import os

directory = r'F:\dataset_sri' # Replace with the directory path containing the .m4a files

# Identify the missing file number
missing_file_number = 163

# Get a sorted list of all .m4a files in the directory
all_files = sorted(os.listdir(directory), key=lambda x: int(x.split('.')[0]))

for file in all_files:
    # Extract the file number from the file name
    file_number = int(file.split('.')[0])
    # If the file number is greater than the missing file number, decrement the file number
    if file_number > missing_file_number:
        new_file_number = file_number - 1
        new_file_name = f'{new_file_number}.m4a'
        # Rename the file
        os.rename(os.path.join(directory, file), os.path.join(directory, new_file_name))


In [31]:
from spafe.features.gfcc import gfcc
from spafe.utils import vis
import scipy.io.wavfile as wav

In [32]:
import soundfile as sf
from pydub import AudioSegment
import librosa
import numpy as np 
import os
import fnmatch
import numpy as np
import librosa
import torch # import PyTorch
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [33]:
directory = r'F:\dataset_sri'
output_directory= r'F:\dains'
os.makedirs(output_directory, exist_ok=True)

wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

def extract_mfcc(file_name):
    # Convert m4a file to wav            
    audio = AudioSegment.from_file(file_name)
    audio.export("temp.wav", format="wav")
    data, samplerate = sf.read("temp.wav")
    
    mfccs = librosa.feature.mfcc(y=data, sr=samplerate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
    
    return mfccs_processed




 # replace with your output directory

# Ensure the output directory exists


for file in wav_files:
    mfccs = extract_mfcc(file)
    
    # Convert array to torch tensor
    mfccs = torch.tensor(mfccs)

    # Construct output file name by replacing .wav with .pt and changing the directory
    output_file = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '.pt'))
    
    # Save to a PyTorch file (.pt)
    torch.save(mfccs, output_file)

In [34]:
import os

directory = r'F:\dains' # Replace with the directory path containing the .m4a files

# Identify the missing file number
missing_file_number = 163

# Get a sorted list of all .m4a files in the directory
all_files = sorted(os.listdir(directory), key=lambda x: int(x.split('.')[0]))

for file in all_files:
    # Extract the file number from the file name
    file_number = int(file.split('.')[0])
    # If the file number is greater than the missing file number, decrement the file number
    if file_number > missing_file_number:
        new_file_number = file_number - 1
        new_file_name = f'{new_file_number}.pt'
        # Rename the file
        os.rename(os.path.join(directory, file), os.path.join(directory, new_file_name))


In [12]:
import os

directory = r'F:\data' # Replace with the directory path containing the files

In [ ]:
directory = r'F:\dataset_sri'
output_directory= r'F:\dataset_feature'
os.makedirs(output_directory, exist_ok=True)

wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

def extract_mfcc(file_name):
    # Convert m4a file to wav            
    audio = AudioSegment.from_file(file_name)
    audio.export("temp.wav", format="wav")
    data, samplerate = sf.read("temp.wav")
    
    mfccs = librosa.feature.mfcc(y=data, sr=samplerate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
    
    return mfccs_processed




 # replace with your output directory

# Ensure the output directory exists


for file in wav_files:
    mfccs = extract_mfcc(file)
    
    # Convert array to torch tensor
    mfccs = torch.tensor(mfccs)

    # Construct output file name by replacing .wav with .pt and changing the directory
    output_file = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '.pt'))
    
    # Save to a PyTorch file (.pt)
    torch.save(mfccs, output_file)

In [1]:
import os
import torch
import fnmatch
import librosa
import numpy as np
from pydub import AudioSegment

def convert_m4a_to_wav(file_path, output_path):
    audio = AudioSegment.from_file(file_path, format="m4a")
    audio.export(output_path, format="wav")

def extract_melscale_features(file_name):
    y, sr = librosa.load(file_name)
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mels_db = librosa.power_to_db(mels, ref=np.max)
    return mels_db.flatten() # Flatten the feature array

def extract_melspectrogram(file_name):
    y, sr = librosa.load(file_name)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    return spectrogram.flatten() # Flatten the spectrogram

def extract_audio_features(file_name):
    y, sr = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0]
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    # Change here: Find max pitch for each frame and then take their mean as overall pitch
    pitch = np.mean([pitch[magnitude.argmax()] for pitch, magnitude in zip(pitches, magnitudes)])
    fft = np.abs(np.fft.fft(y))[:len(y)//2]

    # Combine features
    combined_features = np.hstack([mfccs.mean(axis=1), spectral_centroids.mean(), spectral_rolloff.mean(), zero_crossing_rate.mean(), pitch, fft.mean()])

    return combined_features

In [5]:
from sklearn.decomposition import PCA

directory = r'F:\dataset_sri'

def extract_and_save_features(file_paths, output_directory, pca_model=None):
    all_features = []

    for file in file_paths:
        # Convert m4a file to wav
        wav_file_path = file.replace('.m4a', '.wav')
        convert_m4a_to_wav(file, wav_file_path)

        # Extract features
        melscale_features = extract_melscale_features(wav_file_path)
        melspectrogram = extract_melspectrogram(wav_file_path)
        audio_features = extract_audio_features(wav_file_path)

        # Combine features
        combined_features = np.concatenate([melscale_features, melspectrogram, audio_features])

        # Append features to list
        all_features.append(combined_features)
        print(file)

    # Stack all features into one 2D array
    all_features = np.vstack(all_features)

    # If a PCA model is not provided, create and fit one
    if pca_model is None:
        pca_model = PCA(n_components=60)
        pca_model.fit(all_features)

    # Use the PCA model to reduce the dimensions of all data
    reduced_features = pca_model.transform(all_features)

    # Save each file's features as a .pt file
    for i, file in enumerate(file_paths):
        # Convert array to torch tensor
        reduced_feature_tensor = torch.tensor(reduced_features[i])

        # Construct output file name by replacing .m4a with .pt and changing the directory
        output_file = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '.pt'))

        # Save to a PyTorch file (.pt)
        torch.save(reduced_feature_tensor, output_file)

    # Return the PCA model in case we want to use the same model for other data
    return pca_model


# Load all .m4a files
wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

# Create output directory if it doesn't exist
output_directory = r'F:\reduced_feature'
os.makedirs(output_directory, exist_ok=True)

# Extract, reduce, and save features
pca_model = extract_and_save_features(wav_files, output_directory)


F:\dataset_sri\0.m4a
F:\dataset_sri\1.m4a
F:\dataset_sri\10.m4a
F:\dataset_sri\100.m4a
F:\dataset_sri\1000.m4a
F:\dataset_sri\1001.m4a
F:\dataset_sri\1002.m4a
F:\dataset_sri\1003.m4a
F:\dataset_sri\1004.m4a
F:\dataset_sri\1005.m4a
F:\dataset_sri\1006.m4a
F:\dataset_sri\1007.m4a
F:\dataset_sri\1008.m4a
F:\dataset_sri\1009.m4a
F:\dataset_sri\101.m4a
F:\dataset_sri\1010.m4a
F:\dataset_sri\1011.m4a
F:\dataset_sri\1012.m4a
F:\dataset_sri\1013.m4a
F:\dataset_sri\1014.m4a
F:\dataset_sri\1015.m4a
F:\dataset_sri\1016.m4a
F:\dataset_sri\1017.m4a
F:\dataset_sri\1018.m4a
F:\dataset_sri\1019.m4a
F:\dataset_sri\102.m4a
F:\dataset_sri\1020.m4a
F:\dataset_sri\1021.m4a
F:\dataset_sri\1022.m4a
F:\dataset_sri\1023.m4a
F:\dataset_sri\1024.m4a
F:\dataset_sri\1025.m4a
F:\dataset_sri\1026.m4a
F:\dataset_sri\1027.m4a
F:\dataset_sri\1028.m4a
F:\dataset_sri\1029.m4a
F:\dataset_sri\103.m4a
F:\dataset_sri\1030.m4a
F:\dataset_sri\1031.m4a
F:\dataset_sri\1032.m4a
F:\dataset_sri\1033.m4a
F:\dataset_sri\1034.m4a
F:\d

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 124946 and the array at index 1 has size 51730

In [113]:
import os
import torch
import fnmatch
import librosa
import numpy as np
from pydub import AudioSegment

def convert_m4a_to_wav(file_path, output_path):
    audio = AudioSegment.from_file(file_path, format="m4a")
    audio.export(output_path, format="wav")

def extract_melscale_features(file_name):
    y, sr = librosa.load(file_name)
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mels_db = librosa.power_to_db(mels, ref=np.max)
    return mels_db.flatten() # Flatten the feature array

def extract_melspectrogram(file_name):
    y, sr = librosa.load(file_name)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    return spectrogram.flatten() # Flatten the spectrogram

def extract_audio_features(file_name):
    y, sr = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)[0]
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    # Change here: Find max pitch for each frame and then take their mean as overall pitch
    pitch = np.mean([pitch[magnitude.argmax()] for pitch, magnitude in zip(pitches, magnitudes)])
    fft = np.abs(np.fft.fft(y))[:len(y)//2]

    # Combine features
    combined_features = np.hstack([mfccs.mean(axis=1), spectral_centroids.mean(), spectral_rolloff.mean(), zero_crossing_rate.mean(), pitch, fft.mean()])

    return combined_features


directory = r'F:\dataset_sri'
output_directory= r'F:\dataset_c_feature'
os.makedirs(output_directory, exist_ok=True)

wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

for file in wav_files:
    # Convert m4a file to wav  
    wav_file_path = file.replace('.m4a', '.wav')
    convert_m4a_to_wav(file, wav_file_path)

    # Extract features
    melscale_features = extract_melscale_features(wav_file_path)
    melspectrogram = extract_melspectrogram(wav_file_path)
    audio_features = extract_audio_features(wav_file_path)

    # Combine features
    combined_features = np.concatenate([melscale_features, melspectrogram, audio_features])

    # Convert array to torch tensor
    combined_features = torch.tensor(combined_features)

    # Construct output file name by replacing .m4a with .pt and changing the directory
    output_file = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '.pt'))
    
    # Save to a PyTorch file (.pt)
    torch.save(combined_features, output_file)


In [13]:
import os

directory = r'F:\data' # Replace with the directory path containing the files  # Replace with the directory path containing the files

start_index = 0  # Starting index for renaming
end_index = 5993  # Ending index for renaming
offset = 2312  # Offset to add to the second range

for filename in os.listdir(directory):
    if filename.endswith(".pt"):
        original_path = os.path.join(directory, filename)

        # Extract the numeric index from the filename
        original_index = int(filename[:-3])

        # Check if the file index falls within the desired range
        if 0 <= original_index <= 2311:
            # Calculate the new index for the first range
            new_index = original_index
        elif 7040 <= original_index <= 10721:
            # Calculate the new index for the second range
            new_index = original_index - 7040 + offset
        else:
            continue

        # Create the new filename
        new_filename = str(new_index) + ".pt"
        new_path = os.path.join(directory, new_filename)

        # Skip renaming if the target file already exists
        if os.path.exists(new_path):
            continue

        # Rename the file
        os.rename(original_path, new_path)


In [36]:
import os


output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(len(data))
print(len(labels))

# Initialize Logistic Regression model
model = LogisticRegression(solver='liblinear')


# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))


5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       711
           1       0.90      0.88      0.89       488

    accuracy                           0.91      1199
   macro avg       0.91      0.91      0.91      1199
weighted avg       0.91      0.91      0.91      1199



In [106]:
from pydub import AudioSegment

def convert_m4a_to_wav(file_path, output_path):
    audio = AudioSegment.from_file(file_path, format="m4a")
    audio.export(output_path, format="wav")

def extract_melscale_features(file_name):
    y, sr = librosa.load(file_name)
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mels_db = librosa.power_to_db(mels, ref=np.max)
    return mels_db.flatten() # Flatten the feature array

def extract_melspectrogram(file_name):
    y, sr = librosa.load(file_name)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    return spectrogram.flatten() # Flatten the spectrogram

def extract_audio_features(file_name):
    y, sr = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

    # Combine features
    combined_features = np.hstack([mfccs.mean(axis=1), spectral_centroids.mean(), spectral_rolloff.mean(), zero_crossing_rate.mean()])

    return combined_features


def extract_embeddings(file_name):
    # Load audio file
    audio, _ = librosa.load(file_name)
    # Resample to 16kHz
    audio = librosa.resample(audio, orig_sr=44100, target_sr=16000)
    # Process with Wav2Vec2 processor
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    # Get embeddings from Wav2Vec2 model
    embeddings = model(input_values).last_hidden_state
    # Average over time dimension to get a single embedding vector
    embeddings = torch.mean(embeddings, dim=1)
    # Convert to numpy array
    embeddings = embeddings.detach().numpy()
    
    return embeddings    


In [107]:
def extract_embeddings(file_name):
    # Load audio file
    audio, _ = librosa.load(file_name)
    # Resample to 16kHz
    audio = librosa.resample(audio, orig_sr=44100, target_sr=16000)
    # Process with Wav2Vec2 processor
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    # Get embeddings from Wav2Vec2 model
    embeddings = model(input_values).last_hidden_state
    # Average over time dimension to get a single embedding vector
    embeddings = torch.mean(embeddings, dim=1)
    # Convert to numpy array
    embeddings = embeddings.detach().numpy()
    
    return embeddings

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import torch
import librosa
from transformers import Wav2Vec2Model, Wav2Vec2Processor

In [109]:
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Extract Mel scale features, Mel spectrogram, embeddings, and other audio features
melscale_features = [extract_melscale_features(file) for file in wav_files]
melspectrograms = [extract_melspectrogram(file) for file in wav_files]
embeddings = [extract_embeddings(file) for file in wav_files]
audio_features = [extract_audio_features(file) for file in wav_files]

# Combine all features
all_features = np.concatenate([melscale_features, embeddings, audio_features], axis=1)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(all_features, labels, test_size=0.2, random_state=42)

# Initialize Logistic Regression model
model = LogisticRegression(solver='liblinear')

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))

In [38]:
import torch
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.metrics import classification_report

In [ ]:
import os

output_directory = r'F:\dains' 

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(len(data))
print(len(labels))


model = svm.SVC(kernel='linear')  # linear kernel

# Train the model



# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))


5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       711
           1       0.91      0.88      0.90       488

    accuracy                           0.92      1199
   macro avg       0.91      0.91      0.91      1199
weighted avg       0.92      0.92      0.92      1199



In [40]:
import os


output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print(len(data))
print(len(labels))

# Initialize Logistic Regression model
model = LogisticRegression(solver='liblinear')


# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))


5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      1069
           1       0.90      0.88      0.89       729

    accuracy                           0.91      1798
   macro avg       0.91      0.91      0.91      1798
weighted avg       0.91      0.91      0.91      1798



In [41]:
import os


output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
print(len(data))
print(len(labels))

# Initialize Logistic Regression model
model = LogisticRegression(solver='liblinear')


# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))


5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       900
           1       0.90      0.89      0.89       599

    accuracy                           0.92      1499
   macro avg       0.91      0.91      0.91      1499
weighted avg       0.92      0.92      0.92      1499



In [42]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



output_directory = r'F:\dains' 

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print(len(data))
print(len(labels))


model = RandomForestClassifier(n_estimators=100)  # Use 100 decision trees

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1069
           1       0.89      0.87      0.88       729

    accuracy                           0.91      1798
   macro avg       0.90      0.90      0.90      1798
weighted avg       0.91      0.91      0.91      1798



In [43]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



output_directory = r'F:\dains' 

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)
print(len(data))
print(len(labels))


model = RandomForestClassifier(n_estimators=100)  # Use 100 decision trees

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       900
           1       0.88      0.87      0.88       599

    accuracy                           0.90      1499
   macro avg       0.90      0.90      0.90      1499
weighted avg       0.90      0.90      0.90      1499



In [44]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



output_directory = r'F:\dains' 

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)
print(len(data))
print(len(labels))


model = RandomForestClassifier(n_estimators=100)  # Use 100 decision trees

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      1445
           1       0.88      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.90      0.89      0.89      2398
weighted avg       0.90      0.90      0.90      2398



In [45]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



output_directory = r'F:\dains' 

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(len(data))
print(len(labels))


model = RandomForestClassifier(n_estimators=100)  # Use 100 decision trees

# Train the model
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

# Print performance metrics
print(classification_report(y_test, y_pred))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       711
           1       0.88      0.87      0.87       488

    accuracy                           0.90      1199
   macro avg       0.89      0.89      0.89      1199
weighted avg       0.90      0.90      0.90      1199



In [46]:
import numpy as np
import torch
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
import os

output_directory = r'F:\dains' 

# Initialize data and labels list
data = []
labels = []

# Load extracted features
# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print(len(data))
print(len(labels))

skf = StratifiedKFold(n_splits=3)
# Initialize Logistic Regression model
model = LogisticRegression(solver='liblinear')

# model = svm.SVC(kernel='linear')  # linear kernel

# Perform cross-validation
accuracies = []
for train_index, test_index in skf.split(data, labels):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Test the model
    y_pred = model.predict(X_test)
    
    # Print performance metrics
    print(classification_report(y_test, y_pred))
    
    # Compute accuracy and add it to the list
    accuracies.append(accuracy_score(y_test, y_pred))

# Compute and print the average accuracy
print('Average accuracy:', np.mean(accuracies))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      1227
           1       0.90      0.89      0.90       771

    accuracy                           0.92      1998
   macro avg       0.92      0.92      0.92      1998
weighted avg       0.92      0.92      0.92      1998

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      1228
           1       0.90      0.89      0.90       770

    accuracy                           0.92      1998
   macro avg       0.92      0.92      0.92      1998
weighted avg       0.92      0.92      0.92      1998

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1227
           1       0.90      0.87      0.89       770

    accuracy                           0.91      1997
   macro avg       0.91      0.91      0.91      1997
weighted avg       0.91      0.91      0.91      1997


In [16]:
import numpy as np
import torch
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
import os

output_directory = r'F:\dains' 

# Initialize data and labels list
data = []
labels = []

# Load extracted features
# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print(len(data))
print(len(labels))

skf = StratifiedKFold(n_splits=10)
# Initialize Logistic Regression model


model = svm.SVC(kernel='linear')  # linear kernel

# Perform cross-validation
accuracies = []
for train_index, test_index in skf.split(data, labels):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Test the model
    y_pred = model.predict(X_test)
    
    # Print performance metrics
    print(classification_report(y_test, y_pred))
    
    # Compute accuracy and add it to the list
    accuracies.append(accuracy_score(y_test, y_pred))

# Compute and print the average accuracy
print('Average accuracy:', np.mean(accuracies))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       368
           1       0.87      0.90      0.89       232

    accuracy                           0.91       600
   macro avg       0.90      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       369
           1       0.92      0.89      0.91       231

    accuracy                           0.93       600
   macro avg       0.93      0.92      0.93       600
weighted avg       0.93      0.93      0.93       600

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       369
           1       0.93      0.86      0.89       231

    accuracy                           0.92       600
   macro avg       0.92      0.91      0.91       600
weighted avg       0.92      0.92      0.92       600


KeyboardInterrupt: 

In [15]:
import numpy as np
import torch
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
import os

output_directory = r'F:\dains' 

# Initialize data and labels list
data = []
labels = []

# Load extracted features
# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    # Assign labels: first 11 files are female, next 11 files are male
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)
print(len(data))
print(len(labels))

# Split data into train and test sets
# Ensure the number of samples are consistent between `data` and `labels`
assert len(data) == len(labels), "Mismatched sample sizes: {} vs {}".format(len(data), len(labels))

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
print(len(data))
print(len(labels))

skf = StratifiedKFold(n_splits=10)
# Initialize Logistic Regression model

model = LogisticRegression(solver='liblinear')
# linear kernel

# Perform cross-validation
accuracies = []
for train_index, test_index in skf.split(data, labels):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Test the model
    y_pred = model.predict(X_test)
    
    # Print performance metrics
    print(classification_report(y_test, y_pred))
    
    # Compute accuracy and add it to the list
    accuracies.append(accuracy_score(y_test, y_pred))

# Compute and print the average accuracy
print('Average accuracy:', np.mean(accuracies))

5993
5993
5993
5993
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       368
           1       0.87      0.91      0.89       232

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       369
           1       0.92      0.89      0.91       231

    accuracy                           0.93       600
   macro avg       0.93      0.92      0.92       600
weighted avg       0.93      0.93      0.93       600

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       369
           1       0.92      0.87      0.89       231

    accuracy                           0.92       600
   macro avg       0.92      0.91      0.91       600
weighted avg       0.92      0.92      0.92       600


In [31]:
! pip install keras

                                              0.0/1.7 MB ? eta -:--:--
     ----                                     0.2/1.7 MB 3.5 MB/s eta 0:00:01
     ---------                                0.4/1.7 MB 5.1 MB/s eta 0:00:01
     ---------------                          0.7/1.7 MB 5.2 MB/s eta 0:00:01
     ---------------------                    0.9/1.7 MB 5.4 MB/s eta 0:00:01
     ---------------------------              1.2/1.7 MB 5.4 MB/s eta 0:00:01
     ----------------------------------       1.5/1.7 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------  1.7/1.7 MB 5.5 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 5.3 MB/s eta 0:00:00


In [33]:
! pip install tensorflow

                                              0.0/272.8 MB ? eta -:--:--
                                              0.2/272.8 MB 6.1 MB/s eta 0:00:45
                                              0.6/272.8 MB 7.6 MB/s eta 0:00:36
                                              1.0/272.8 MB 8.3 MB/s eta 0:00:33
                                              1.5/272.8 MB 8.7 MB/s eta 0:00:32
                                              1.9/272.8 MB 8.8 MB/s eta 0:00:31
                                              2.4/272.8 MB 9.0 MB/s eta 0:00:31
                                              2.8/272.8 MB 9.1 MB/s eta 0:00:30
                                              3.3/272.8 MB 9.1 MB/s eta 0:00:30
                                              3.8/272.8 MB 9.2 MB/s eta 0:00:30
                                              4.2/272.8 MB 9.2 MB/s eta 0:00:30
                                              4.7/272.8 MB 9.3 MB/s eta 0:00:29
                                              5

In [95]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define the directory
directory = r'F:\dains'

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)

data = np.array(data)
labels = np.array(labels)

# Reshape the input data to match the CNN input shape (n_samples, height, width, channels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


class SelfAttention(tf.keras.layers.Layer):
    def _init_(self):
        super(SelfAttention, self)._init_()

    def build(self, input_shape):
        self.query = Dense(input_shape[-1])
        self.key = Dense(input_shape[-1])
        self.value = Dense(input_shape[-1])
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)

        attended_features = tf.matmul(attention_weights, value)

        return attended_features
        
# Define the model
def get_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 1), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=(3, 1), activation='softmax', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
   
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Reshape((64, -1)))  # Reshape for Self-Attention
    model.add(SelfAttention())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Changed to softmax for multi-class

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and train the model
cnn_model = get_cnn_model()
cnn_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
cnn_score = cnn_model.evaluate(X_test, y_test, verbose=0)
print('CNN Test loss:', cnn_score[0])
print('CNN Test accuracy:', cnn_score[1])

Epoch 1/10
150/150 [==============================] - 9s 26ms/step - loss: 0.6435 - accuracy: 0.6249 - val_loss: 0.5833 - val_accuracy: 0.6622
Epoch 2/10
150/150 [==============================] - 2s 16ms/step - loss: 0.5106 - accuracy: 0.7520 - val_loss: 0.3799 - val_accuracy: 0.8365
Epoch 3/10
150/150 [==============================] - 2s 16ms/step - loss: 0.4479 - accuracy: 0.7977 - val_loss: 0.3753 - val_accuracy: 0.8440
Epoch 4/10
150/150 [==============================] - 2s 13ms/step - loss: 0.4019 - accuracy: 0.8177 - val_loss: 0.3042 - val_accuracy: 0.8891
Epoch 5/10
150/150 [==============================] - 2s 15ms/step - loss: 0.3890 - accuracy: 0.8306 - val_loss: 0.3680 - val_accuracy: 0.8465
Epoch 6/10
150/150 [==============================] - 2s 14ms/step - loss: 0.3800 - accuracy: 0.8360 - val_loss: 0.3025 - val_accuracy: 0.8749
Epoch 7/10
150/150 [==============================] - 2s 15ms/step - loss: 0.3494 - accuracy: 0.8517 - val_loss: 0.3059 - val_accuracy: 0.8724

In [105]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Dense, Dropout, TimeDistributed
from keras.layers import Layer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import backend as K

directory = r'F:\dains'
data = []
labels = []

# Load features and labels
for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)  # Assuming 2310 is the threshold between the classes

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Attention Mechanism
class Attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        if self.return_sequences:
            return output
        return K.sum(output, axis=1)

# Define the DNN model
def get_model(input_shape, output_units):
    model = Sequential()
    model.add(TimeDistributed(Dense(units=64, activation='relu'), input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(Attention(return_sequences=True))  # Add Attention layer after TimeDistributed Dense
    model.add(Dropout(0.5))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_units, activation='softmax'))  # Use softmax for multi-class classification

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and train the model
model = get_model(X_train.shape[1:], np.unique(labels).size)
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\Dell\AppData\Roaming\Python\Python39\site-pack

In [104]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Flatten, TimeDistributed
from keras.layers import Layer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import backend as K

# Directory containing the .pt files
directory = r'F:\dains'

data = []
labels = []

# Load features and labels
for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)  # Assuming 2310 is the threshold between the classes

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Attention Mechanism
class Attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention,self).__init__()
        
    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="zeros")        
        super(Attention,self).build(input_shape)
        
    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:
            return output
        return K.sum(output, axis=1)

# Define the LSTM model
def get_model(input_shape, output_units):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Attention(return_sequences=True))  # Add Attention layer after LSTM
    model.add(Dropout(0.5))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(output_units, activation='softmax'))  # Use softmax for multi-class classification

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and train the model
model = get_model(X_train.shape[1:], np.unique(labels).size)
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
150/150 [==============================] - 14s 52ms/step - loss: 0.5227 - accuracy: 0.7428 - val_loss: 0.4743 - val_accuracy: 0.7898
Epoch 2/10
150/150 [==============================] - 8s 53ms/step - loss: 0.4492 - accuracy: 0.7972 - val_loss: 0.4471 - val_accuracy: 0.7932
Epoch 3/10
150/150 [==============================] - 8s 51ms/step - loss: 0.4358 - accuracy: 0.8106 - val_loss: 0.4371 - val_accuracy: 0.8123
Epoch 4/10
150/150 [==============================] - 8s 54ms/step - loss: 0.4260 - accuracy: 0.8150 - val_loss: 0.4447 - val_accuracy: 0.8057
Epoch 5/10
150/150 [==============================] - 9s 57ms/step - loss: 0.4194 - accuracy: 0.8204 - val_loss: 0.4093 - val_accuracy: 0.8165
Epoch 6/10
150/150 [==============================] - 16s 104ms/step - loss: 0.4020 - accuracy: 0.8298 - val_loss: 0.3968 - val_accuracy: 0.8207
Epoch 7/10
150/150 [==============================] - 10s 64ms/step - loss: 0.3915 - accuracy: 0.8319 - val_loss: 0.4037 - val_accuracy: 0.

In [100]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
import tensorflow as tf

# Define the directory
directory = r'F:\dains'

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)

data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

n_components = 10  # The size of the reduced data, tweak this as needed

nmf = NMF(n_components=n_components, init='random', random_state=0)

# Flatten your data and apply NMF
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))

X_train_flat = X_train_flat - np.min(X_train_flat)
X_test_flat = X_test_flat - np.min(X_test_flat)

X_train_nmf = nmf.fit_transform(X_train_flat)
X_test_nmf = nmf.transform(X_test_flat)

# Now reshape your data back into 2D
X_train = X_train_nmf.reshape(-1, n_components, 1, 1)
X_test = X_test_nmf.reshape(-1, n_components, 1, 1)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

class SelfAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def build(self, input_shape):
        self.query = Dense(input_shape[-1])
        self.key = Dense(input_shape[-1])
        self.value = Dense(input_shape[-1])
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)

        attended_features = tf.matmul(attention_weights, value)

        return attended_features
        
# Define the model
def get_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 1), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Reshape((-1, 64)))  # Reshape for Self-Attention
    model.add(SelfAttention())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Changed to softmax for multi-class

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and train the model
cnn_model = get_cnn_model()
cnn_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
cnn_score = cnn_model.evaluate(X_test, y_test, verbose=0)
print('CNN Test loss:', cnn_score[0])
print('CNN Test accuracy:', cnn_score[1])


c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Epoch 1/10
150/150 [==============================] - 3s 7ms/step - loss: 0.7406 - accuracy: 0.5668 - val_loss: 0.6761 - val_accuracy: 0.5930
Epoch 2/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6718 - accuracy: 0.6097 - val_loss: 0.6749 - val_accuracy: 0.5930
Epoch 3/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6661 - accuracy: 0.6133 - val_loss: 0.6716 - val_accuracy: 0.5930
Epoch 4/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6620 - accuracy: 0.6218 - val_loss: 0.6687 - val_accuracy: 0.5930
Epoch 5/10
150/150 [==============================] - 1s 5ms/step - loss: 0.6593 - accuracy: 0.6164 - val_loss: 0.6629 - val_accuracy: 0.5930
Epoch 6/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6565 - accuracy: 0.6204 - val_loss: 0.6576 - val_accuracy: 0.5930
Epoch 7/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6499 - accuracy: 0.6256 - val_loss: 0.6504 - val_accuracy: 0.5930
Epoch 

In [39]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\data'  # Replace with the directory path containing the .pt files

data = []
labels = []

# Load the MFCC features and corresponding labels from .pt files
# Load the MFCC features and corresponding labels from .pt files
for filename in os.listdir(directory):
    if filename.endswith(".pt"):
        feature = torch.load(os.path.join(directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns

        data.append(feature.view(1, -1).numpy())
        labels.append(int(filename[:-3]))  # Extract the label from the filename


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

    

Shape of the first reshaped item in data: (1, 40)


In [ ]:
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...




# Create the CNN model


model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [86]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []



for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy())
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Reshape the input data to match the CNN input shape
input_shape = (data[0].shape[1], 1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], 1)
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Now you can define and train your CNN as before
# ...






def get_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(data[0].shape[0], data[0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

cnn_model = get_cnn_model()
cnn_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
cnn_score = cnn_model.evaluate(X_test, y_test, verbose=0)
print('CNN Test loss:', cnn_score[0])
print('CNN Test accuracy:', cnn_score[1])



Shape of the first reshaped item in data: (1, 40)


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d. Consider increasing the input size. Received input shape [None, 1, 40, 1] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

directory = r'F:\data'  # Replace with the directory path containing the .pt files



data = []
labels = []

for i in range(5994):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(output_directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2311 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# The input shape will now be (40, 1)
input_shape = (data[0].shape[0], data[0].shape[1])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)





kfold = StratifiedKFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(data, labels):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    # Convert labels to one-hot encoding
    y_train_fold = keras.utils.to_categorical(labels[train], num_classes)
    y_test_fold = keras.utils.to_categorical(labels[test], num_classes)

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(data[train], y_train_fold, batch_size=32, epochs=10, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(data[test], y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    fold_no = fold_no + 1




In [17]:
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

directory = r'F:\dains'  # Replace with the directory path containing the .pt files



data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(output_directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# The input shape will now be (40, 1)
input_shape = (data[0].shape[0], data[0].shape[1])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)





kfold = StratifiedKFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(data, labels):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    # Convert labels to one-hot encoding
    y_train_fold = keras.utils.to_categorical(labels[train], num_classes)
    y_test_fold = keras.utils.to_categorical(labels[test], num_classes)

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(data[train], y_train_fold, batch_size=32, epochs=10, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(data[test], y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    fold_no = fold_no + 1




Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
169/169 [==============================] - 2s 6ms/step - loss: 0.3505 - accuracy: 0.8561
Epoch 2/10
169/169 [==============================] - 1s 7ms/step - loss: 0.2643 - accuracy: 0.8995
Epoch 3/10
169/169 [==============================] - 1s 6ms/step - loss: 0.2451 - accuracy: 0.9077
Epoch 4/10
169/169 [==============================] - 1s 6ms/step - loss: 0.2182 - accuracy: 0.9182
Epoch 5/10
169/169 [==============================] - 1s 6ms/step - loss: 0.2044 - accuracy: 0.9240
Epoch 6/10
169/169 [==============================] - 1s 6ms/step - loss: 0.1964 - accuracy: 0.9262
Epoch 7/10
169/169 [==============================] - 1s 6ms/step - loss: 0.1873 - accuracy: 0.9301
Epoch 8/10
169/169 [==============================] - 1s 5ms/step - loss: 0.1866 - accuracy: 0.9290
Epoch 9/10
169/169 [=======

In [1]:
# Initialize total and count to keep track of total accuracy and number of samples
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

directory = r'F:\dains'  # Replace with the directory path containing the .pt files



data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# The input shape will now be (40, 1)
input_shape = (data[0].shape[0], data[0].shape[1])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)





kfold = StratifiedKFold(n_splits=10, shuffle=True)


total_acc = 0
count = 0

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(data, labels):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    # Convert labels to one-hot encoding
    y_train_fold = keras.utils.to_categorical(labels[train], num_classes)
    y_test_fold = keras.utils.to_categorical(labels[test], num_classes)

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(data[train], y_train_fold, batch_size=32, epochs=10, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(data[test], y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    # Keep track of total accuracy and number of samples
    total_acc += scores[1] * len(test)
    count += len(test)

    fold_no = fold_no + 1

# Compute the overall weighted accuracy
overall_acc = total_acc / count
print(f'Overall accuracy: {overall_acc*100}%')


NameError: name 'output_directory' is not defined

In [18]:
# Initialize total and count to keep track of total accuracy and number of samples
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

directory = r'F:\dains'  # Replace with the directory path containing the .pt files



data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(output_directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# The input shape will now be (40, 1)
input_shape = (data[0].shape[0], data[0].shape[1])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)





kfold = StratifiedKFold(n_splits=10, shuffle=True)


total_acc = 0
count = 0

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(data, labels):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    # Convert labels to one-hot encoding
    y_train_fold = keras.utils.to_categorical(labels[train], num_classes)
    y_test_fold = keras.utils.to_categorical(labels[test], num_classes)

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(data[train], y_train_fold, batch_size=32, epochs=10, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(data[test], y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    # Keep track of total accuracy and number of samples
    total_acc += scores[1] * len(test)
    count += len(test)

    fold_no = fold_no + 1

# Compute the overall weighted accuracy
overall_acc = total_acc / count
print(f'Overall accuracy: {overall_acc*100}%')


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
169/169 [==============================] - 1s 6ms/step - loss: 0.3911 - accuracy: 0.8481
Epoch 2/10
169/169 [==============================] - 1s 6ms/step - loss: 0.2460 - accuracy: 0.9080
Epoch 3/10
169/169 [==============================] - 1s 5ms/step - loss: 0.2242 - accuracy: 0.9141
Epoch 4/10
169/169 [==============================] - 1s 6ms/step - loss: 0.2161 - accuracy: 0.9206
Epoch 5/10
169/169 [==============================] - 1s 5ms/step - loss: 0.2014 - accuracy: 0.9247
Epoch 6/10
169/169 [==============================] - 1s 5ms/step - loss: 0.1931 - accuracy: 0.9319
Epoch 7/10
169/169 [==============================] - 1s 5ms/step - loss: 0.1842 - accuracy: 0.9299
Epoch 8/10
169/169 [==============================] - 1s 5ms/step - loss: 0.1695 - accuracy: 0.9381
Epoch 9/10
169/169 [=======

In [2]:
# Initialize total and count to keep track of total accuracy and number of samples
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

directory = r'F:\dains'  # Replace with the directory path containing the .pt files



data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# The input shape will now be (40, 1)
input_shape = (data[0].shape[0], data[0].shape[1])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)





kfold = StratifiedKFold(n_splits=3, shuffle=True)


total_acc = 0
count = 0

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(data, labels):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    # Convert labels to one-hot encoding
    y_train_fold = keras.utils.to_categorical(labels[train], num_classes)
    y_test_fold = keras.utils.to_categorical(labels[test], num_classes)

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(data[train], y_train_fold, batch_size=32, epochs=10, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(data[test], y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    # Keep track of total accuracy and number of samples
    total_acc += scores[1] * len(test)
    count += len(test)

    fold_no = fold_no + 1

# Compute the overall weighted accuracy
overall_acc = total_acc / count
print(f'Overall accuracy: {overall_acc*100}%')


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
125/125 [==============================] - 1s 6ms/step - loss: 0.4408 - accuracy: 0.8265
Epoch 2/10
125/125 [==============================] - 1s 6ms/step - loss: 0.2544 - accuracy: 0.8989
Epoch 3/10
125/125 [==============================] - 1s 5ms/step - loss: 0.2181 - accuracy: 0.9181
Epoch 4/10
125/125 [==============================] - 1s 6ms/step - loss: 0.2031 - accuracy: 0.9242
Epoch 5/10
125/125 [==============================] - 1s 5ms/step - loss: 0.1914 - accuracy: 0.9247
Epoch 6/10
125/125 [==============================] - 0s 3ms/step - loss: 0.1777 - accuracy: 0.9332
Epoch 7/10
125/125 [==============================] - 0s 4ms/step - loss: 0.1716 - accuracy: 0.9334
Epoch 8/10
125/125 [==============================] - 0s 4ms/step - loss: 0.1672 - accuracy: 0.9387
Epoch 9/10
125/125 [=======

In [55]:
# Initialize total and count to keep track of total accuracy and number of samples
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

directory = r'F:\data'  # Replace with the directory path containing the .pt files



data = []
labels = []

for i in range(5994):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(output_directory, filename))
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2311 else 0)


labels = np.array(labels)
data  = np.array(data)

# Now you should be able to get a 3D shape from each element in data
print("Shape of the first reshaped item in data:", data[0].shape)

# Split the data into training and testing sets
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# The input shape will now be (40, 1)
input_shape = (data[0].shape[0], data[0].shape[1])

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)





kfold = StratifiedKFold(n_splits=100, shuffle=True)


total_acc = 0
count = 0

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(data, labels):
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    # Convert labels to one-hot encoding
    y_train_fold = keras.utils.to_categorical(labels[train], num_classes)
    y_test_fold = keras.utils.to_categorical(labels[test], num_classes)

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(data[train], y_train_fold, batch_size=32, epochs=10, verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(data[test], y_test_fold, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    # Keep track of total accuracy and number of samples
    total_acc += scores[1] * len(test)
    count += len(test)

    fold_no = fold_no + 1

# Compute the overall weighted accuracy
overall_acc = total_acc / count
print(f'Overall accuracy: {overall_acc*100}%')


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5994,)
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
186/186 [==============================] - 1s 4ms/step - loss: 0.3789 - accuracy: 0.8519
Epoch 2/10
186/186 [==============================] - 1s 4ms/step - loss: 0.2460 - accuracy: 0.9068
Epoch 3/10
186/186 [==============================] - 1s 4ms/step - loss: 0.2238 - accuracy: 0.9183
Epoch 4/10
186/186 [==============================] - 1s 4ms/step - loss: 0.2061 - accuracy: 0.9232
Epoch 5/10
186/186 [==============================] - 1s 4ms/step - loss: 0.1995 - accuracy: 0.9238
Epoch 6/10
186/186 [==============================] - 1s 4ms/step - loss: 0.1880 - accuracy: 0.9296
Epoch 7/10
186/186 [==============================] - 1s 4ms/step - loss: 0.1742 - accuracy: 0.9356
Epoch 8/10
186/186 [==============================] - 1s 4ms/step - loss: 0.1634 - accuracy: 0.9415
Epoch 9/10
186/186 [=======

In [19]:
import os
import numpy as np
import torch
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))  # Corrected the path to the directory
        # Reshape the feature to a 2D array with a single row and 40 columns
        data.append(feature.view(1, -1).numpy()[0])  # Adjusted this line to flatten the data
        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

# Define and fit the Gaussian Mixture Model
gmm = GaussianMixture(n_components=num_classes, random_state=42)
gmm.fit(X_train)

# Predict the labels for the test data
y_pred = gmm.predict(X_test)

# Calculate accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy:', accuracy)


Test accuracy: 0.27697441601779754


In [61]:
! pip install --upgrade tensorflow
! pip install --upgrade keras


In [ ]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Flatten

In [20]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)

def get_model(op_nodes):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=32, return_sequences=True), input_shape=(data[0].shape[0], data[0].shape[1])))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.1))
    model.add(Dense(op_nodes, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create the BiLSTM model
model = get_model(num_classes)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 14s 52ms/step - loss: 0.4694 - accuracy: 0.7833 - val_loss: 0.4316 - val_accuracy: 0.7990
Epoch 2/10
150/150 [==============================] - 6s 41ms/step - loss: 0.4252 - accuracy: 0.8150 - val_loss: 0.4200 - val_accuracy: 0.8157
Epoch 3/10
150/150 [==============================] - 6s 42ms/step - loss: 0.3936 - accuracy: 0.8304 - val_loss: 0.3967 - val_accuracy: 0.8265
Epoch 4/10
150/150 [==============================] - 7s 45ms/step - loss: 0.3730 - accuracy: 0.8423 - val_loss: 0.3881 - val_accuracy: 0.8265
Epoch 5/10
150/150 [==============================] - 7s 47ms/step - loss: 0.3635 - accuracy: 0.8431 - val_loss: 0.3808 - val_accuracy: 0.8332
Epoch 6/10
150/150 [==============================] - 7s 48ms/step - loss: 0.3527 - accuracy: 0.8479 - val_loss: 0.3635 - val_accuracy: 0.8390
Epoch 7/10
150/150 [==============================] - 7s 44ms/ste

In [103]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

# Directory containing the .pt files
directory = r'F:\dains'

data = []
labels = []

# Load features and labels
for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)  # Assuming 2310 is the threshold between the classes

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the LSTM model
def get_model(input_shape, output_units):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(LSTM(units=64))
    model.add(Dropout(0.5))
    model.add(Dense(output_units, activation='softmax'))  # Use softmax for multi-class classification

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create and train the model
model = get_model(X_train.shape[1:], np.unique(labels).size)
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
150/150 [==============================] - 11s 47ms/step - loss: 0.4800 - accuracy: 0.7839 - val_loss: 0.4546 - val_accuracy: 0.8032
Epoch 2/10
150/150 [==============================] - 7s 45ms/step - loss: 0.4428 - accuracy: 0.8083 - val_loss: 0.4476 - val_accuracy: 0.8048
Epoch 3/10
150/150 [==============================] - 7s 46ms/step - loss: 0.4250 - accuracy: 0.8181 - val_loss: 0.4305 - val_accuracy: 0.8107
Epoch 4/10
150/150 [==============================] - 7s 45ms/step - loss: 0.4094 - accuracy: 0.8214 - val_loss: 0.4024 - val_accuracy: 0.8307
Epoch 5/10
150/150 [==============================] - 7s 48ms/step - loss: 0.4025 - accuracy: 0.8283 - val_loss: 0.4000 - val_accuracy: 0.8165
Epoch 6/10
150/150 [==============================] - 7s 47ms/step - loss: 0.3893 - accuracy: 0.8310 - val_loss: 0.3860 - val_accuracy: 0.8307
Epoch 7/10
150/150 [==============================] - 7s 47ms/step - loss: 0.3826 - accuracy: 0.8327 - val_loss: 0.3741 - val_accuracy: 0.831

In [72]:
from keras.layers import Flatten, Dense
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels:", labels.shape)  # It should be (n_samples, n_classes)

# Define the model creation function
def create_model():
    model = Sequential()
    model.add(Flatten(input_shape=(data[0].shape[0], 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper for use in scikit-learn
model = KerasClassifier(build_fn=create_model)

# Define the hyperparameters to tune and their respective values
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20],
}

# Perform grid search using cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

print("Best Hyperparameters:", best_params)

# Calculate accuracy using score method
accuracy = best_model.score(X_test, y_test)
print('Test accuracy:', accuracy)


Shape of the first reshaped item in data: (40, 1)
Shape of labels: (5993,)
Epoch 1/10


C:\Users\Dell\AppData\Local\Temp\ipykernel_11672\439863856.py:42: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


100/100 [==============================] - 1s 3ms/step - loss: 0.8198 - accuracy: 0.7860
Epoch 2/10
100/100 [==============================] - 0s 4ms/step - loss: 0.3103 - accuracy: 0.8849
Epoch 3/10
100/100 [==============================] - 0s 3ms/step - loss: 0.2888 - accuracy: 0.8927
Epoch 4/10
100/100 [==============================] - 0s 3ms/step - loss: 0.2350 - accuracy: 0.9111
Epoch 5/10
100/100 [==============================] - 0s 3ms/step - loss: 0.2545 - accuracy: 0.9030
Epoch 6/10
100/100 [==============================] - 0s 4ms/step - loss: 0.2277 - accuracy: 0.9149
Epoch 7/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2948 - accuracy: 0.8942
Epoch 8/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2294 - accuracy: 0.9246
Epoch 9/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2445 - accuracy: 0.9168
Epoch 10/10
50/50 [==============================] - 0s 2ms/step - loss: 0.2643 - accuracy: 0.9074
Epoch 1/10
1

In [75]:
from keras.layers import Flatten, Dense
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from keras.wrappers.scikit_learn import KerasClassifier

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

# Reshape data to have 2 dimensions
data = data.reshape(data.shape[0], -1)

# Apply feature selection
selector = SelectKBest(f_classif, k='all')  # Adjust k as needed
data_selected = selector.fit_transform(data, labels)

# Apply feature ensemble
ensemble_data = np.concatenate((data, data_selected), axis=1)

X_train, X_test, y_train, y_test = train_test_split(ensemble_data, labels, test_size=0.2, random_state=42)

print("Shape of labels:", labels.shape)  # It should be (n_samples, n_classes)

# Define the model creation function
def create_model():
    model = Sequential()
    model.add(Flatten(input_shape=(ensemble_data.shape[1], 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier wrapper for use in scikit-learn
model = KerasClassifier(build_fn=create_model)

# Define the hyperparameters to tune and their respective values
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20],
}

# Perform grid search using cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

print("Best Hyperparameters:", best_params)

# Apply feature reduction with PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

# Create a voting classifier with the best model and reduced features
classifiers = [('best_model', best_model)]
voting_classifier = VotingClassifier(classifiers, voting='soft')

# Train the voting classifier on the reduced features
voting_classifier.fit(X_train_reduced, y_train)

# Calculate accuracy on the reduced features
accuracy = voting_classifier.score(X_test_reduced, y_test)
print('Test accuracy:', accuracy)


Shape of the first reshaped item in data: (40, 1)
Shape of labels: (5993,)
Epoch 1/10


C:\Users\Dell\AppData\Local\Temp\ipykernel_11672\277589820.py:55: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


100/100 [==============================] - 1s 2ms/step - loss: 0.6465 - accuracy: 0.8364
Epoch 2/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2963 - accuracy: 0.8967
Epoch 3/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2865 - accuracy: 0.8967
Epoch 4/10
100/100 [==============================] - 0s 3ms/step - loss: 0.2729 - accuracy: 0.9046
Epoch 5/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2623 - accuracy: 0.9105
Epoch 6/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2653 - accuracy: 0.9115
Epoch 7/10
100/100 [==============================] - 0s 2ms/step - loss: 0.2439 - accuracy: 0.9074
Epoch 8/10
100/100 [==============================] - 0s 4ms/step - loss: 0.2288 - accuracy: 0.9180
Epoch 9/10
100/100 [==============================] - 0s 3ms/step - loss: 0.2319 - accuracy: 0.9152
Epoch 10/10
50/50 [==============================] - 0s 2ms/step - loss: 0.3408 - accuracy: 0.8942
Epoch 1/10
1

ValueError: The estimator KerasClassifier should be a classifier.

In [10]:
from keras.layers import Flatten, Dense
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)
def get_model():
    model = Sequential()
    model.add(Flatten(input_shape=(data[0].shape[0], 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = get_model()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 2s 5ms/step - loss: 0.4819 - accuracy: 0.8471 - val_loss: 0.3191 - val_accuracy: 0.8882
Epoch 2/10
150/150 [==============================] - 1s 3ms/step - loss: 0.2786 - accuracy: 0.8986 - val_loss: 0.2692 - val_accuracy: 0.8941
Epoch 3/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2523 - accuracy: 0.9099 - val_loss: 0.2547 - val_accuracy: 0.9033
Epoch 4/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2838 - accuracy: 0.9013 - val_loss: 0.2535 - val_accuracy: 0.9058
Epoch 5/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2480 - accuracy: 0.9141 - val_loss: 0.2816 - val_accuracy: 0.9033
Epoch 6/10
150/150 [==============================] - 0s 3ms/step - loss: 0.2938 - accuracy: 0.8978 - val_loss: 0.2855 - val_accuracy: 0.8999
Epoch 7/10
150/150 [==============================] - 1s 5ms/step - loss

In [49]:
from keras.layers import Flatten, Dense
from keras.models import Model
import os
import numpy as np
import torch
from sklearn import svm
from sklearn.model_selection import train_test_split
from keras.layers import Flatten, Dense
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


# Defining CNN model
def get_cnn_model():
    inputs = keras.Input(shape=(data[0].shape[0], 1))
    x = keras.layers.Conv1D(32, 3, activation='relu')(inputs)
    x = keras.layers.MaxPooling1D(2)(x)
    x = keras.layers.Conv1D(64, 3, activation='relu')(x)
    x = keras.layers.MaxPooling1D(2)(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(128)(x)  # this layer will be used as the feature extractor
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Training CNN
cnn_model = get_cnn_model()
cnn_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

# Extracting features
X_train_features = cnn_model.predict(X_train)
X_test_features = cnn_model.predict(X_test)

# Training SVM on extracted features
svm_model = svm.SVC(probability=True)  # probability=True enables probability estimates
svm_model.fit(X_train_features, y_train)

# Evaluating SVM
score = svm_model.score(X_test_features, y_test)
print('Test accuracy:', score)

# Predicting probabilities
y_test_probs = svm_model.predict_proba(X_test_features)
print('Test probabilities:', y_test_probs)


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 2s 6ms/step - loss: 5.6764
Epoch 2/10
150/150 [==============================] - 1s 6ms/step - loss: 5.3523
Epoch 3/10
150/150 [==============================] - 1s 6ms/step - loss: 5.3089
Epoch 4/10
150/150 [==============================] - 1s 6ms/step - loss: 5.2942
Epoch 5/10
150/150 [==============================] - 1s 5ms/step - loss: 5.1531
Epoch 6/10
150/150 [==============================] - 1s 6ms/step - loss: 5.2710
Epoch 7/10
150/150 [==============================] - 1s 7ms/step - loss: 5.4589
Epoch 8/10
150/150 [==============================] - 1s 7ms/step - loss: 5.7336
Epoch 9/10
150/150 [==============================] - 1s 6ms/step - loss: 5.8009
Epoch 10/10
38/38 [==============================] - 0s 3ms/step
Test accuracy: 0.8590492076730609
Test probabilities: [[0.95730611 0.04269389]
 [0.12249191 0.87750809]
 [0.5        0.5       ]
 

In [50]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


class SelfAttention(tf.keras.layers.Layer):
    def _init_(self):
        super(SelfAttention, self)._init_()

    def build(self, input_shape):
        self.query = Dense(input_shape[-1])
        self.key = Dense(input_shape[-1])
        self.value = Dense(input_shape[-1])
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)

        attended_features = tf.matmul(attention_weights, value)

        return attended_features


def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(data[0].shape[0], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((64, -1)))  # Reshape for Self-Attention
    model.add(SelfAttention())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = get_model()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 4s 14ms/step - loss: 0.7657 - accuracy: 0.7920 - val_loss: 0.3881 - val_accuracy: 0.8440
Epoch 2/10
150/150 [==============================] - 2s 11ms/step - loss: 0.3503 - accuracy: 0.8567 - val_loss: 0.3598 - val_accuracy: 0.8582
Epoch 3/10
150/150 [==============================] - 2s 11ms/step - loss: 0.2988 - accuracy: 0.8821 - val_loss: 0.3314 - val_accuracy: 0.8641
Epoch 4/10
150/150 [==============================] - 2s 11ms/step - loss: 0.2972 - accuracy: 0.8821 - val_loss: 0.3244 - val_accuracy: 0.8624
Epoch 5/10
150/150 [==============================] - 2s 10ms/step - loss: 0.2639 - accuracy: 0.9001 - val_loss: 0.2794 - val_accuracy: 0.8849
Epoch 6/10
150/150 [==============================] - 2s 11ms/step - loss: 0.2663 - accuracy: 0.9001 - val_loss: 0.3059 - val_accuracy: 0.8832
Epoch 7/10
150/150 [==============================] - 2s 11ms/step

In [76]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


class SelfAttention(tf.keras.layers.Layer):
    def _init_(self):
        super(SelfAttention, self)._init_()

    def build(self, input_shape):
        self.query = Dense(input_shape[-1])
        self.key = Dense(input_shape[-1])
        self.value = Dense(input_shape[-1])
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)

        attended_features = tf.matmul(attention_weights, value)

        return attended_features


def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(data[0].shape[0], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((64, -1)))  # Reshape for Self-Attention
    model.add(SelfAttention())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = get_model()
model.fit(X_train, y_train, batch_size=8, epochs=20, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/20
600/600 [==============================] - 6s 6ms/step - loss: 0.5143 - accuracy: 0.8164 - val_loss: 0.3821 - val_accuracy: 0.8474
Epoch 2/20
600/600 [==============================] - 4s 7ms/step - loss: 0.3482 - accuracy: 0.8584 - val_loss: 0.3234 - val_accuracy: 0.8716
Epoch 3/20
600/600 [==============================] - 4s 7ms/step - loss: 0.3051 - accuracy: 0.8813 - val_loss: 0.3278 - val_accuracy: 0.8749
Epoch 4/20
600/600 [==============================] - 4s 7ms/step - loss: 0.2834 - accuracy: 0.8874 - val_loss: 0.2842 - val_accuracy: 0.8957
Epoch 5/20
600/600 [==============================] - 4s 6ms/step - loss: 0.2615 - accuracy: 0.8995 - val_loss: 0.2707 - val_accuracy: 0.8932
Epoch 6/20
600/600 [==============================] - 4s 6ms/step - loss: 0.2549 - accuracy: 0.9047 - val_loss: 0.2691 - val_accuracy: 0.8907
Epoch 7/20
600/600 [==============================] - 4s 6ms/step - loss

In [80]:
import os
import numpy as np
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


class SelfAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def build(self, input_shape):
        self.query = Dense(input_shape[-1])
        self.key = Dense(input_shape[-1])
        self.value = Dense(input_shape[-1])
        super(SelfAttention, self).build(input_shape)

    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)

        attended_features = tf.matmul(attention_weights, value)

        return attended_features


# Apply feature selection
k = 30  # Adjust the number of selected features as needed, less than 40
selector = SelectKBest(f_classif, k=k)
X_train_selected = selector.fit_transform(X_train.reshape(X_train.shape[0], -1), y_train)
X_test_selected = selector.transform(X_test.reshape(X_test.shape[0], -1))

# Apply feature reduction
n_components = 20  # Adjust the number of reduced components as needed, less than k
pca = PCA(n_components=n_components)
X_train_reduced = pca.fit_transform(X_train_selected)
X_test_reduced = pca.transform(X_test_selected)


def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(n_components, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((64, -1)))  # Reshape for Self-Attention
    model.add(SelfAttention())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = get_model()
model.fit(X_train_reduced.reshape(X_train_reduced.shape[0], X_train_reduced.shape[1], 1), y_train, batch_size=32, epochs=20, verbose=1, validation_data=(X_test_reduced.reshape(X_test_reduced.shape[0], X_test_reduced.shape[1], 1), y_test))
score = model.evaluate(X_test_reduced.reshape(X_test_reduced.shape[0], X_test_reduced.shape[1], 1), y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/20
150/150 [==============================] - 3s 7ms/step - loss: 0.4375 - accuracy: 0.7887 - val_loss: 0.4752 - val_accuracy: 0.7973
Epoch 2/20
150/150 [==============================] - 1s 7ms/step - loss: 0.3089 - accuracy: 0.8763 - val_loss: 0.3161 - val_accuracy: 0.8841
Epoch 3/20
150/150 [==============================] - 1s 6ms/step - loss: 0.2747 - accuracy: 0.8934 - val_loss: 0.3021 - val_accuracy: 0.8832
Epoch 4/20
150/150 [==============================] - 1s 7ms/step - loss: 0.2641 - accuracy: 0.8957 - val_loss: 0.3117 - val_accuracy: 0.8866
Epoch 5/20
150/150 [==============================] - 1s 7ms/step - loss: 0.2500 - accuracy: 0.8999 - val_loss: 0.3037 - val_accuracy: 0.8682
Epoch 6/20
150/150 [==============================] - 1s 7ms/step - loss: 0.2344 - accuracy: 0.9070 - val_loss: 0.3173 - val_accuracy: 0.8882
Epoch 7/20
150/150 [==============================] - 1s 7ms/step - loss

In [78]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


class AttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_heads, num_layers):
        super(AttentionModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        self.attention = nn.MultiheadAttention(hidden_size, num_heads)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # (seq_len, batch_size, input_size)
        x, _ = self.attention(x, x, x)  # self-attention
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x



def get_dataloader(X, y, batch_size):
    dataset = TensorDataset(torch.Tensor(X), torch.Tensor(y))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader


def train(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    correct = 0
    total_samples = 0

    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        predicted = (outputs > 0.5).squeeze().long()
        correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct / total_samples

    return average_loss, accuracy


def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)

            total_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).squeeze().long()
            correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct / total_samples

    return average_loss, accuracy


# Model parameters
input_size = data[0].shape[0]
hidden_size = 64
num_heads = 4
num_layers = 1
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Create the model
model = AttentionModel(input_size, hidden_size, num_heads, num_layers)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create dataloaders
train_dataloader = get_dataloader(X_train, y_train, batch_size)
test_dataloader = get_dataloader(X_test, y_test, batch_size)

# Training loop
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_dataloader, criterion, optimizer)
    test_loss, test_accuracy = evaluate(model, test_dataloader, criterion)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print()

Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1280x1 and 40x64)

In [70]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape, Concatenate
from tensorflow.keras.layers import GlobalAveragePooling1D
directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, head_dim):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.total_dim = num_heads * head_dim

        self.query = Dense(self.total_dim)
        self.key = Dense(self.total_dim)
        self.value = Dense(self.total_dim)

        self.concatenation = Concatenate(axis=-1)
        self.attention_weights = None

    def build(self, input_shape):
        super(MultiHeadSelfAttention, self).build(input_shape)

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]

        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        query = tf.reshape(query, (batch_size, -1, self.num_heads, self.head_dim))
        query = tf.transpose(query, perm=[0, 2, 1, 3])

        key = tf.reshape(key, (batch_size, -1, self.num_heads, self.head_dim))
        key = tf.transpose(key, perm=[0, 2, 1, 3])

        value = tf.reshape(value, (batch_size, -1, self.num_heads, self.head_dim))
        value = tf.transpose(value, perm=[0, 2, 1, 3])

        attention_weights = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)

        self.attention_weights = attention_weights

        attended_features = tf.matmul(attention_weights, value)
        attended_features = tf.transpose(attended_features, perm=[0, 2, 1, 3])
        attended_features = tf.reshape(attended_features, (batch_size, -1, self.total_dim))

        return attended_features




def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(data[0].shape[0], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((64, -1)))  # Reshape for Multi-Head Self-Attention
    model.summary()  # Summary after adding Reshape layer
    print(model.output_shape)  # Print the output shape after the Reshape layer
    
    model.add(MultiHeadSelfAttention(num_heads=12, head_dim=128))
    model.add(GlobalAveragePooling1D())  # Add this line
    model.summary()  # Summary after adding MultiHeadSelfAttention layer
    print(model.output_shape)  # Print the output shape after the MultiHeadSelfAttention layer
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = get_model()
model.summary()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_36 (Conv1D)          (None, 38, 32)            128       
                                                                 
 max_pooling1d_30 (MaxPoolin  (None, 19, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_37 (Conv1D)          (None, 17, 64)            6208      
                                                                 
 max_pooling1d_31 (MaxPoolin  (None, 8, 64)            0         
 g1D)                                                            
                                                                 
 flatten_23 (Flatten)        (None, 512)               0         
                                           

In [61]:
from keras.layers import Flatten, Dense
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


def get_model():
    model = Sequential()
    model.add(Flatten(input_shape=(data[0].shape[0], 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = get_model()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 1s 4ms/step - loss: 0.5530 - accuracy: 0.8348 - val_loss: 0.2687 - val_accuracy: 0.8982
Epoch 2/10
150/150 [==============================] - 1s 4ms/step - loss: 0.3148 - accuracy: 0.8894 - val_loss: 0.2740 - val_accuracy: 0.9074
Epoch 3/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2858 - accuracy: 0.9009 - val_loss: 0.2412 - val_accuracy: 0.9074
Epoch 4/10
150/150 [==============================] - 1s 5ms/step - loss: 0.2862 - accuracy: 0.9005 - val_loss: 0.2665 - val_accuracy: 0.9008
Epoch 5/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2948 - accuracy: 0.8959 - val_loss: 0.4442 - val_accuracy: 0.8407
Epoch 6/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2713 - accuracy: 0.9099 - val_loss: 0.2439 - val_accuracy: 0.9099
Epoch 7/10
150/150 [==============================] - 1s 4ms/step - loss

In [44]:
!pip install tensorflow --upgrade

^C


In [58]:
pip install tensorflow_addons

                                              0.0/746.7 kB ? eta -:--:--
     -----                                  112.6/746.7 kB 2.2 MB/s eta 0:00:01
     ---------                              194.6/746.7 kB 2.0 MB/s eta 0:00:01
     --------------                         276.5/746.7 kB 1.9 MB/s eta 0:00:01
     ------------------                     358.4/746.7 kB 2.0 MB/s eta 0:00:01
     ----------------------                 440.3/746.7 kB 2.0 MB/s eta 0:00:01
     ---------------------------            532.5/746.7 kB 2.0 MB/s eta 0:00:01
     -------------------------------        624.6/746.7 kB 2.0 MB/s eta 0:00:01
     -----------------------------------    706.6/746.7 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 746.7/746.7 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [59]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Reshape
import tensorflow_addons as tfa

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)


class SelfAttention(tf.keras.layers.Layer):
    def _init_(self):
        super(SelfAttention, self)._init_()
        self.query = Dense(self.units)
        self.key = Dense(self.units)
        self.value = Dense(self.units)
        self.multihead_attention = tfa.layers.MultiHeadAttention(num_heads=8, key_dim=self.units)

    def call(self, inputs):
        query = self.query(inputs)
        key = self.key(inputs)
        value = self.value(inputs)

        attention_weights = self.multihead_attention(query, key, value)
        attended_features = tf.reduce_mean(attention_weights, axis=1)

        return attended_features


def get_model():
    model = Sequential()
    model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(data[0].shape[0], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Reshape((64, -1)))  # Reshape for Self-Attention
    model.add(SelfAttention())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = get_model()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


AssertionError: Duplicate registrations for type 'experimentalOptimizer'

In [46]:
from keras.layers import Flatten, Dense, Attention, GlobalAveragePooling1D
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)

def get_model():
    model = Sequential()
    model.add(GlobalAveragePooling1D(input_shape=(data[0].shape[0], 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Attention())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = get_model()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0.pt
1.pt
2.pt
3.pt
4.pt
5.pt
6.pt
7.pt
8.pt
9.pt
10.pt
11.pt
12.pt
13.pt
14.pt
15.pt
16.pt
17.pt
18.pt
19.pt
20.pt
21.pt
22.pt
23.pt
24.pt
25.pt
26.pt
27.pt
28.pt
29.pt
30.pt
31.pt
32.pt
33.pt
34.pt
35.pt
36.pt
37.pt
38.pt
39.pt
40.pt
41.pt
42.pt
43.pt
44.pt
45.pt
46.pt
47.pt
48.pt
49.pt
50.pt
51.pt
52.pt
53.pt
54.pt
55.pt
56.pt
57.pt
58.pt
59.pt
60.pt
61.pt
62.pt
63.pt
64.pt
65.pt
66.pt
67.pt
68.pt
69.pt
70.pt
71.pt
72.pt
73.pt
74.pt
75.pt
76.pt
77.pt
78.pt
79.pt
80.pt
81.pt
82.pt
83.pt
84.pt
85.pt
86.pt
87.pt
88.pt
89.pt
90.pt
91.pt
92.pt
93.pt
94.pt
95.pt
96.pt
97.pt
98.pt
99.pt
100.pt
101.pt
102.pt
103.pt
104.pt
105.pt
106.pt
107.pt
108.pt
109.pt
110.pt
111.pt
112.pt
113.pt
114.pt
115.pt
116.pt
117.pt
118.pt
119.pt
120.pt
121.pt
122.pt
123.pt
124.pt
125.pt
126.pt
127.pt
128.pt
129.pt
130.pt
131.pt
132.pt
133.pt
134.pt
135.pt
136.pt
137.pt
138.pt
139.pt
140.pt
141.pt
142.pt
143.pt
144.pt
145.pt
146.pt
147.pt
148.pt
149.pt
150.pt
151.pt
152.pt
153.pt
154.pt
155.pt
156.pt
157.pt
158.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



5852.pt
5853.pt
5854.pt
5855.pt
5856.pt
5857.pt
5858.pt
5859.pt
5860.pt
5861.pt
5862.pt
5863.pt
5864.pt
5865.pt
5866.pt
5867.pt
5868.pt
5869.pt
5870.pt
5871.pt
5872.pt
5873.pt
5874.pt
5875.pt
5876.pt
5877.pt
5878.pt
5879.pt
5880.pt
5881.pt
5882.pt
5883.pt
5884.pt
5885.pt
5886.pt
5887.pt
5888.pt
5889.pt
5890.pt
5891.pt
5892.pt
5893.pt
5894.pt
5895.pt
5896.pt
5897.pt
5898.pt
5899.pt
5900.pt
5901.pt
5902.pt
5903.pt
5904.pt
5905.pt
5906.pt
5907.pt
5908.pt
5909.pt
5910.pt
5911.pt
5912.pt
5913.pt
5914.pt
5915.pt
5916.pt
5917.pt
5918.pt
5919.pt
5920.pt
5921.pt
5922.pt
5923.pt
5924.pt
5925.pt
5926.pt
5927.pt
5928.pt
5929.pt
5930.pt
5931.pt
5932.pt
5933.pt
5934.pt
5935.pt
5936.pt
5937.pt
5938.pt
5939.pt
5940.pt
5941.pt
5942.pt
5943.pt
5944.pt
5945.pt
5946.pt
5947.pt
5948.pt
5949.pt
5950.pt
5951.pt
5952.pt
5953.pt
5954.pt
5955.pt
5956.pt
5957.pt
5958.pt
5959.pt
5960.pt
5961.pt
5962.pt
5963.pt
5964.pt
5965.pt
5966.pt
5967.pt
5968.pt
5969.pt
5970.pt
5971.pt
5972.pt
5973.pt
5974.pt
5975.pt
5976.pt


ValueError: Exception encountered when calling layer "attention" (type Attention).

Attention layer must be called on a list of inputs, namely [query, value] or [query, value, key]. Received: Tensor("Placeholder:0", shape=(None, 64), dtype=float32).

Call arguments received by layer "attention" (type Attention):
  • inputs=tf.Tensor(shape=(None, 64), dtype=float32)
  • mask=None
  • training=None
  • return_attention_scores=False
  • use_causal_mask=False

In [47]:
from keras.layers import Input, Dense, Attention, GlobalAveragePooling1D
from keras.models import Model
import os
import numpy as np
import torch
from keras.layers import Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        data.append(feature.view(-1, 1).numpy())
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)

input_layer = Input(shape=(data[0].shape[0], 1))
x = GlobalAveragePooling1D()(input_layer)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Apply self-attention
attention = Attention()([x, x])

output = Dense(1, activation='sigmoid')(attention)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0.pt
1.pt
2.pt
3.pt
4.pt
5.pt
6.pt
7.pt
8.pt
9.pt
10.pt
11.pt
12.pt
13.pt
14.pt
15.pt
16.pt
17.pt
18.pt
19.pt
20.pt
21.pt
22.pt
23.pt
24.pt
25.pt
26.pt
27.pt
28.pt
29.pt
30.pt
31.pt
32.pt
33.pt
34.pt
35.pt
36.pt
37.pt
38.pt
39.pt
40.pt
41.pt
42.pt
43.pt
44.pt
45.pt
46.pt
47.pt
48.pt
49.pt
50.pt
51.pt
52.pt
53.pt
54.pt
55.pt
56.pt
57.pt
58.pt
59.pt
60.pt
61.pt
62.pt
63.pt
64.pt
65.pt
66.pt
67.pt
68.pt
69.pt
70.pt
71.pt
72.pt
73.pt
74.pt
75.pt
76.pt
77.pt
78.pt
79.pt
80.pt
81.pt
82.pt
83.pt
84.pt
85.pt
86.pt
87.pt
88.pt
89.pt
90.pt
91.pt
92.pt
93.pt
94.pt
95.pt
96.pt
97.pt
98.pt
99.pt
100.pt
101.pt
102.pt
103.pt
104.pt
105.pt
106.pt
107.pt
108.pt
109.pt
110.pt
111.pt
112.pt
113.pt
114.pt
115.pt
116.pt
117.pt
118.pt
119.pt
120.pt
121.pt
122.pt
123.pt
124.pt
125.pt
126.pt
127.pt
128.pt
129.pt
130.pt
131.pt
132.pt
133.pt
134.pt
135.pt
136.pt
137.pt
138.pt
139.pt
140.pt
141.pt
142.pt
143.pt
144.pt
145.pt
146.pt
147.pt
148.pt
149.pt
150.pt
151.pt
152.pt
153.pt
154.pt
155.pt
156.pt
157.pt
158.

In [43]:
from keras.layers import Flatten, Dense, Input, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense, Attention,Multiply
from sklearn.model_selection import train_test_split
directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        print(filename)
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the autoencoder model
input_layer = Input(shape=(data[0].shape[0], 1))

x = Conv1D(32, 3, activation="relu", padding="same")(input_layer) # 32 dimensions
x = MaxPooling1D(2, padding="same")(x)
x = Conv1D(1, 3, activation="relu", padding="same")(x)
encoded = MaxPooling1D(2, padding="same")(x)

x = Conv1D(1, 3, activation="relu", padding="same")(encoded)
x = UpSampling1D(2)(x)
x = Conv1D(32, 3, activation="relu", padding="same")(x)
x = UpSampling1D(2)(x)
decoded = Conv1D(1, 3, activation="sigmoid", padding="same")(x)

# Create the autoencoder model
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse")

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, validation_data=(X_test, X_test))

# Use the encoder part of the autoencoder to encode the train and test data
encoder = Model(input_layer, encoded)
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# Now use the encoded data to train a model with an attention layer
inputs = Input(shape=(X_train_encoded.shape[1], X_train_encoded.shape[2]))
attention_probs = Dense(X_train_encoded.shape[1], activation='softmax', name='attention_vec')(inputs)
attention_mul = Multiply(name='attention_mul')([inputs, attention_probs])
out = Flatten()(attention_mul)
out = Dense(1, activation='sigmoid')(out)

attention_model = Model(inputs=inputs, outputs=out)
attention_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
attention_model.fit(X_train_encoded, y_train, batch_size=32, epochs=3, verbose=1, validation_data=(X_test_encoded, y_test))

score = attention_model.evaluate(X_test_encoded, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


0.pt
1.pt
2.pt
3.pt
4.pt
5.pt
6.pt
7.pt
8.pt
9.pt
10.pt
11.pt
12.pt
13.pt
14.pt
15.pt
16.pt
17.pt
18.pt
19.pt
20.pt
21.pt
22.pt
23.pt
24.pt
25.pt
26.pt
27.pt
28.pt
29.pt
30.pt
31.pt
32.pt
33.pt
34.pt
35.pt
36.pt
37.pt
38.pt
39.pt
40.pt
41.pt
42.pt
43.pt
44.pt
45.pt
46.pt
47.pt
48.pt
49.pt
50.pt
51.pt
52.pt
53.pt
54.pt
55.pt
56.pt
57.pt
58.pt
59.pt
60.pt
61.pt
62.pt
63.pt
64.pt
65.pt
66.pt
67.pt
68.pt
69.pt
70.pt
71.pt
72.pt
73.pt
74.pt
75.pt
76.pt
77.pt
78.pt
79.pt
80.pt
81.pt
82.pt
83.pt
84.pt
85.pt
86.pt
87.pt
88.pt
89.pt
90.pt
91.pt
92.pt
93.pt
94.pt
95.pt
96.pt
97.pt
98.pt
99.pt
100.pt
101.pt
102.pt
103.pt
104.pt
105.pt
106.pt
107.pt
108.pt
109.pt
110.pt
111.pt
112.pt
113.pt
114.pt
115.pt
116.pt
117.pt
118.pt
119.pt
120.pt
121.pt
122.pt
123.pt
124.pt
125.pt
126.pt
127.pt
128.pt
129.pt
130.pt
131.pt
132.pt
133.pt
134.pt
135.pt
136.pt
137.pt
138.pt
139.pt
140.pt
141.pt
142.pt
143.pt
144.pt
145.pt
146.pt
147.pt
148.pt
149.pt
150.pt
151.pt
152.pt
153.pt
154.pt
155.pt
156.pt
157.pt
158.

In [11]:
from keras.layers import Flatten, Dense
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)
def get_model():
    model = Sequential()
    model.add(Flatten(input_shape=(data[0].shape[0], 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = get_model()
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Shape of the first reshaped item in data: (40, 1)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 2s 5ms/step - loss: 0.9579 - accuracy: 0.8089 - val_loss: 0.3570 - val_accuracy: 0.8432
Epoch 2/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2587 - accuracy: 0.9015 - val_loss: 0.2496 - val_accuracy: 0.8991
Epoch 3/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2599 - accuracy: 0.9018 - val_loss: 0.2820 - val_accuracy: 0.8999
Epoch 4/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2419 - accuracy: 0.9105 - val_loss: 0.2405 - val_accuracy: 0.9091
Epoch 5/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2389 - accuracy: 0.9105 - val_loss: 0.3115 - val_accuracy: 0.8707
Epoch 6/10
150/150 [==============================] - 1s 4ms/step - loss: 0.2388 - accuracy: 0.9122 - val_loss: 0.2407 - val_accuracy: 0.9099
Epoch 7/10
150/150 [==============================] - 1s 5ms/step - loss

In [ ]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\data'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5994):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1, 1).numpy())  # Adjusted this line
        labels.append(1 if i <= 2311 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)

def get_model(op_nodes):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=32, return_sequences=True), input_shape=(data[0].shape[0], data[0].shape[1])))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.1))
    model.add(Dense(op_nodes, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Create the BiLSTM model
model = get_model(num_classes)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [4]:
import os
import numpy as np
import torch
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split

In [15]:
import os
import numpy as np
import torch
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split

# Load data
directory = r'F:\data'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5994):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        
        data.append(feature.view(40,1).numpy()) # Adjusted this line to match ResNet input
        labels.append(1 if i <= 2311 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

num_classes = len(np.unique(labels))

# Define ResNet block




Shape of the first item in data: (40, 1)


In [21]:
import os
import numpy as np
import torch
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split

# Load data
directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(40,1).numpy())  # Adjusted this line to match ResNet input
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)

print("Shape of the first item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

num_classes = len(np.unique(labels))

# Define ResNet block
def resnet_block(inputs, num_filters, kernel_size, strides, activation='relu'):
    x = layers.Conv1D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    x = layers.Conv1D(num_filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    shortcut = layers.Conv1D(num_filters, kernel_size=1, strides=strides, padding='same')(inputs)
    shortcut = layers.BatchNormalization()(shortcut)
    x = layers.add([x, shortcut])
    if activation:
        x = layers.Activation(activation)(x)
    return x

# Define ResNet model
def resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv1D(16, kernel_size=3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    
    x = resnet_block(x, 32, 3, 2)
    x = resnet_block(x, 32, 3, 1)
    
    x = resnet_block(x, 64, 3, 2)
    x = resnet_block(x, 64, 3, 1)
    
    x = layers.GlobalAveragePooling1D()(x)
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model


model = resnet(input_shape=(40, 1), num_classes=num_classes)

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Shape of the first item in data: (40, 1)


In [22]:
model = resnet(input_shape=(40, 1), num_classes=num_classes)

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
150/150 [==============================] - 9s 13ms/step - loss: 0.4173 - accuracy: 0.8175 - val_loss: 0.7338 - val_accuracy: 0.6188
Epoch 2/10
150/150 [==============================] - 2s 11ms/step - loss: 0.3365 - accuracy: 0.8623 - val_loss: 0.4569 - val_accuracy: 0.7898
Epoch 3/10
150/150 [==============================] - 2s 10ms/step - loss: 0.2968 - accuracy: 0.8782 - val_loss: 0.3351 - val_accuracy: 0.8582
Epoch 4/10
150/150 [==============================] - 2s 11ms/step - loss: 0.2731 - accuracy: 0.8861 - val_loss: 0.4321 - val_accuracy: 0.8332
Epoch 5/10
150/150 [==============================] - 2s 11ms/step - loss: 0.2599 - accuracy: 0.8959 - val_loss: 0.2882 - val_accuracy: 0.8816
Epoch 6/10
150/150 [==============================] - 2s 11ms/step - loss: 0.2490 - accuracy: 0.9001 - val_loss: 0.3062 - val_accuracy: 0.8749
Epoch 7/10
150/150 [==============================] - 2s 12ms/step - loss: 0.2363 - accuracy: 0.9059 - val_loss: 0.2967 - val_accuracy: 0.8757

In [27]:
import os

directory = r'F:\dataset_sri' # Replace with the directory path containing the files  # Replace with the directory path containing the files

start_index = 0  # Starting index for renaming
end_index = 5993  # Ending index for renaming
offset = 2312  # Offset to add to the second range

for filename in os.listdir(directory):
    if filename.endswith(".m4a"):
        original_path = os.path.join(directory, filename)

        # Extract the numeric index from the filename
        original_index = int(filename[:-4])

        # Check if the file index falls within the desired range
        if 0 <= original_index <= 2311:
            # Calculate the new index for the first range
            new_index = original_index
        elif 7040 <= original_index <= 10721:
            # Calculate the new index for the second range
            new_index = original_index - 7040 + offset
        else:
            continue

        # Create the new filename
        new_filename = str(new_index) + ".m4a"
        new_path = os.path.join(directory, new_filename)

        # Skip renaming if the target file already exists
        if os.path.exists(new_path):
            continue

        # Rename the file
        os.rename(original_path, new_path)


In [1]:
import os
import numpy as np
import pandas as pd
import scipy.stats
import librosa
import scipy.signal
import soundfile as sf
from pydub import AudioSegment
from scipy.fftpack import fft

directory = r'F:\dataset_sri'  # Replace with the directory path containing the .m4a files
labels = []

features = []

for i in range(5993):
    filename = f'{i}.m4a'
    print(filename)

    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        # Convert m4a file to wav
        audio = AudioSegment.from_file(os.path.join(directory, filename))
        audio.export("temp.wav", format="wav")
        y, sr = sf.read("temp.wav")

        # Perform STFT
        stft = np.abs(librosa.stft(y))

        # Calculate various statistical measures
        meanfreq = np.mean(stft)
        sd = np.std(stft)
        median = np.median(stft)
        Q25 = np.quantile(stft, 0.25)
        Q50 = np.quantile(stft, 0.50)
        Q75 = np.quantile(stft, 0.75)
        IQR = scipy.stats.iqr(stft)
        skewness = scipy.stats.skew(stft.flatten())
        kurtosis = scipy.stats.kurtosis(stft.flatten())

        # Spectral Entropy
        freqs, psd = scipy.signal.welch(y, sr)
        sp_ent = -np.sum(psd*np.log2(psd))

        # Spectral Flatness
        sfm = np.mean(librosa.feature.spectral_flatness(y=y))

        # Mode (Most frequent element)
        mode = scipy.stats.mode(stft.flatten())[0][0]

        # Spectral Centroid
        centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))

        # Meanfun, Minfun, Maxfun
        f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
        meanfun = np.mean(f0)
        minfun = np.min(f0)
        maxfun = np.max(f0)
        
        # Meandom, Mindom, Maxdom, Dfrange
        psd = np.abs(fft(y))**2
        freqs = np.fft.fftfreq(len(psd),1/sr)
        idx = np.where(freqs >= 0)
        freq_weighted = np.sum(psd[idx] * freqs[idx])
        freq = np.sum(psd[idx])
        meandom = freq_weighted / freq
        mindom = np.min(freqs)
        maxdom = np.max(freqs)
        dfrange = maxdom - mindom

        # Modindx
        diffs = np.diff(f0)
        diffs = diffs[~np.isnan(diffs)]
        modindx = np.sum(abs(diffs)) / len(diffs) if len(diffs) > 0 else 0

        # Save feature to the features list
        features.append([meanfreq, sd, median, Q25, Q50, Q75, IQR, skewness, kurtosis, sp_ent, sfm, mode, centroid, meanfun, minfun, maxfun, meandom, mindom, maxdom, dfrange, modindx])

        # Assign labels: first 2312 files are labelled 1, rest are labelled 0
        labels.append(1 if i <= 2310 else 0)

# Convert features and labels to a pandas DataFrame and save to CSV
features_df = pd.DataFrame(features, columns=['meanfreq', 'sd', 'median', 'Q25', 'Q50', 'Q75', 'IQR', 'skewness', 'kurtosis', 'sp_ent', 'sfm', 'mode', 'centroid','meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx'])
labels_df = pd.DataFrame(labels, columns=['label'])
data_df = pd.concat([features_df, labels_df], axis=1)
data_df.to_csv('audio_features10.csv', index=False)



0.m4a


C:\Users\Dell\AppData\Local\Temp\ipykernel_11984\1260300349.py:48: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = scipy.stats.mode(stft.flatten())[0][0]


1.m4a
2.m4a
3.m4a
4.m4a
5.m4a
6.m4a
7.m4a
8.m4a
9.m4a
10.m4a
11.m4a
12.m4a
13.m4a
14.m4a
15.m4a
16.m4a
17.m4a
18.m4a
19.m4a
20.m4a
21.m4a
22.m4a
23.m4a
24.m4a
25.m4a
26.m4a
27.m4a
28.m4a
29.m4a
30.m4a
31.m4a
32.m4a
33.m4a
34.m4a
35.m4a
36.m4a
37.m4a
38.m4a
39.m4a
40.m4a
41.m4a
42.m4a
43.m4a
44.m4a
45.m4a
46.m4a
47.m4a
48.m4a
49.m4a
50.m4a
51.m4a
52.m4a
53.m4a
54.m4a
55.m4a
56.m4a
57.m4a
58.m4a
59.m4a
60.m4a
61.m4a
62.m4a
63.m4a
64.m4a
65.m4a
66.m4a
67.m4a
68.m4a
69.m4a
70.m4a
71.m4a
72.m4a
73.m4a
74.m4a
75.m4a
76.m4a
77.m4a
78.m4a
79.m4a
80.m4a
81.m4a
82.m4a
83.m4a
84.m4a
85.m4a
86.m4a
87.m4a
88.m4a
89.m4a
90.m4a
91.m4a
92.m4a
93.m4a
94.m4a
95.m4a
96.m4a
97.m4a
98.m4a
99.m4a
100.m4a
101.m4a
102.m4a
103.m4a
104.m4a
105.m4a
106.m4a
107.m4a
108.m4a
109.m4a
110.m4a
111.m4a
112.m4a
113.m4a
114.m4a
115.m4a
116.m4a
117.m4a
118.m4a
119.m4a
120.m4a
121.m4a
122.m4a
123.m4a
124.m4a
125.m4a
126.m4a
127.m4a
128.m4a
129.m4a
130.m4a
131.m4a
132.m4a
133.m4a
134.m4a
135.m4a
136.m4a
137.m4a
138.m4a
139.

In [18]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('voice.csv')

# Replace 'male' with 0 and 'female' with 1
df['label'] = df['label'].map({'male': 0, 'female': 1})

# Save the updated dataframe to a new CSV file
df.to_csv('updated_voice.csv', index=False)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])]
labels = df['label']

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(features_train, labels_train)

# Predict on the testing data
labels_pred = model.predict(features_test)

# Measure accuracy of the model
accuracy = accuracy_score(labels_test, labels_pred)
print(f'Accuracy: {accuracy * 100}%')


Accuracy: 75.72977481234362%


In [19]:


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read the data from the CSV file
df = pd.read_csv('updated_voice.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])]
labels = df['label']

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(features_train, labels_train)

# Predict on the testing data
labels_pred = model.predict(features_test)

# Measure accuracy of the model
accuracy = accuracy_score(labels_test, labels_pred)
print(f'Accuracy: {accuracy * 100}%')


Accuracy: 89.1167192429022%


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])]
labels = df['label']

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(features_train, labels_train)

# Predict on the testing data
labels_pred = model.predict(features_test)

# Measure accuracy of the model
accuracy = accuracy_score(labels_test, labels_pred)
print(f'Accuracy: {accuracy * 100}%')


Accuracy: 74.3047830923248%


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read the data from the CSV file
df = pd.read_csv('updated_voice.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])]
labels = df['label']

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(features_train, labels_train)

# Predict on the testing data
labels_pred = model.predict(features_test)

# Measure accuracy of the model
accuracy = accuracy_score(labels_test, labels_pred)
print(f'Accuracy: {accuracy * 100}%')


Accuracy: 89.6950578338591%


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])]
labels = df['label']

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(features_train, labels_train)

# Predict on the testing data
labels_pred = model.predict(features_test)

# Measure accuracy of the model
accuracy = accuracy_score(labels_test, labels_pred)
print(f'Accuracy: {accuracy * 100}%')


Accuracy: 74.84989993328885%


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read the data from the CSV file
# df = pd.read_csv('audio_features10.csv')
df = pd.read_csv('updated_voice.csv')
# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])]
labels = df['label']

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4)

# Create and train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(features_train, labels_train)

# Predict on the testing data
labels_pred = model.predict(features_test)

# Measure accuracy of the model
accuracy = accuracy_score(labels_test, labels_pred)
print(f'Accuracy: {accuracy * 100}%')


Accuracy: 87.93375394321767%


In [6]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=5)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 75.13730835796665%


In [7]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=3)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 74.93736433813216%


In [8]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=7)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 74.83706396548958%


In [9]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=10)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 74.86997774067892%


In [10]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=100)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 75.03785310734463%


In [11]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=200)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 74.8367816091954%


In [12]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=1000)

# Create and train the logistic regression model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, labels):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = LogisticRegression(max_iter=1000)
    model.fit(features_train, labels_train)
    
    # Predict on the testing data
    labels_pred = model.predict(features_test)

    # Measure accuracy of the model
    accuracy = accuracy_score(labels_test, labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Average accuracy: 75.16333333333333%


In [23]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Convert labels to categorical
labels = np_utils.to_categorical(labels)

# Reshape features for LSTM Layer
features = np.reshape(features, (features.shape[0], 1, features.shape[1]))

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=5)

# Create and train the LSTM model within each fold
accuracies = []

def create_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=32, return_sequences=True), input_shape=(1, features.shape[2])))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.2))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

for train_index, test_index in strat_k_fold.split(features, np.argmax(labels, axis=1)):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = create_model()
    model.fit(features_train, labels_train, epochs=10, batch_size=32, verbose=1)

    # Predict on the testing data
    labels_pred = np.argmax(model.predict(features_test), axis=1)

    # Measure accuracy of the model
    accuracy = accuracy_score(np.argmax(labels_test, axis=1), labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Epoch 1/10
150/150 [==============================] - 9s 7ms/step - loss: 0.6709 - accuracy: 0.6143
Epoch 2/10
150/150 [==============================] - 1s 7ms/step - loss: 0.6686 - accuracy: 0.6145
Epoch 3/10
150/150 [==============================] - 1s 7ms/step - loss: 0.6688 - accuracy: 0.6145
Epoch 4/10
150/150 [==============================] - 1s 7ms/step - loss: 0.6678 - accuracy: 0.6145
Epoch 5/10
150/150 [==============================] - 1s 7ms/step - loss: 0.6682 - accuracy: 0.6145
Epoch 6/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6676 - accuracy: 0.6145
Epoch 7/10
150/150 [==============================] - 1s 8ms/step - loss: 0.6674 - accuracy: 0.6145
Epoch 8/10
150/150 [==============================] - 1s 7ms/step - loss: 0.6670 - accuracy: 0.6145
Epoch 9/10
150/150 [==============================] - 1s 7ms/step - loss: 0.6672 - accuracy: 0.6145
Epoch 10/10
38/38 [==============================] - 2s 3ms/step
Epoch 1/10
150/150 [===============

In [22]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import numpy as np

# Read the data from the CSV file
df = pd.read_csv('updated_voice.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Convert labels to categorical
labels = np_utils.to_categorical(labels)

# Reshape features for LSTM Layer
features = np.reshape(features, (features.shape[0], 1, features.shape[1]))

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=5)

# Create and train the LSTM model within each fold
accuracies = []

def create_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=32, return_sequences=True), input_shape=(1, features.shape[2])))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(0.2))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

for train_index, test_index in strat_k_fold.split(features, np.argmax(labels, axis=1)):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]
    
    model = create_model()
    model.fit(features_train, labels_train, epochs=10, batch_size=32, verbose=1)

    # Predict on the testing data
    labels_pred = np.argmax(model.predict(features_test), axis=1)

    # Measure accuracy of the model
    accuracy = accuracy_score(np.argmax(labels_test, axis=1), labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Epoch 1/10
80/80 [==============================] - 12s 5ms/step - loss: 0.6627 - accuracy: 0.6101
Epoch 2/10
80/80 [==============================] - 0s 6ms/step - loss: 0.5813 - accuracy: 0.6910
Epoch 3/10
80/80 [==============================] - 0s 6ms/step - loss: 0.5432 - accuracy: 0.7269
Epoch 4/10
80/80 [==============================] - 1s 6ms/step - loss: 0.5071 - accuracy: 0.7530
Epoch 5/10
80/80 [==============================] - 1s 7ms/step - loss: 0.4720 - accuracy: 0.8070
Epoch 6/10
80/80 [==============================] - 1s 6ms/step - loss: 0.4212 - accuracy: 0.8208
Epoch 7/10
80/80 [==============================] - 0s 6ms/step - loss: 0.3796 - accuracy: 0.8524
Epoch 8/10
80/80 [==============================] - 1s 7ms/step - loss: 0.3595 - accuracy: 0.8650
Epoch 9/10
80/80 [==============================] - 0s 6ms/step - loss: 0.3343 - accuracy: 0.8781
Epoch 10/10
20/20 [==============================] - 2s 3ms/step
Epoch 1/10
80/80 [==============================] - 

In [24]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))


Gradient Boosting Classifier:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91      1445
           1       0.86      0.87      0.87       953

    accuracy                           0.89      2398
   macro avg       0.89      0.89      0.89      2398
weighted avg       0.89      0.89      0.89      2398

Iteration 1, loss = 6.21899637
Iteration 2, loss = 0.42989737
Iteration 3, loss = 0.36013720
Iteration 4, loss = 0.32153972
Iteration 5, loss = 0.29901663
Iteration 6, loss = 0.28151885
Iteration 7, loss = 0.27753715
Iteration 8, loss = 0.26635217
Iteration 9, loss = 0.26465553
Iteration 10, loss = 0.26309853
Iteration 11, loss = 0.25270635
Iteration 12, loss = 0.24163557
Iteration 13, loss = 0.25494582
Iteration 14, loss = 0.25633477
Iteration 15, loss = 0.24788820
Iteration 16, loss = 0.24245390
Iteration 17, loss = 0.23531085
Iteration 18, loss = 0.23448921
Iteration 19, loss = 0.23878744
Iteration 20, loss = 0.24050962
Iteration 2

In [28]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# Custom rule-based ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    predictions = [y_pred_lr[i], y_pred_svm[i], y_pred_rf[i], y_pred_gb[i], y_pred_mlp[i], y_pred_dt[i]]
    if y_test[i] in predictions:
        y_pred_ensemble[i] = y_test[i]
    else:
        y_pred_ensemble[i] = y_pred_mlp[i]

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))




Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      1445
           1       0.89      0.88      0.89       953

    accuracy                           0.91      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.91      0.91      0.91      2398

SVM Classifier:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1445
           1       0.90      0.88      0.89       953

    accuracy                           0.92      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.92      0.92      0.92      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      1445
           1       0.87      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.89      0.89      0.89      2398
weighted

In [101]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("Linear Discriminant Analysis:\n", classification_report(y_test, y_pred_lda))

# KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
print("K-Nearest Neighbour:\n", classification_report(y_test, y_pred_knn))

# Custom rule-based ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    predictions = [y_pred_lr[i], y_pred_svm[i], y_pred_rf[i], y_pred_gb[i], y_pred_mlp[i], y_pred_dt[i], y_pred_lda[i], y_pred_knn[i]]
    if y_test[i] in predictions:
        y_pred_ensemble[i] = y_test[i]
    else:
        y_pred_ensemble[i] = y_pred_mlp[i]  # As an example, we use MLP's prediction as a default

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      1445
           1       0.89      0.88      0.89       953

    accuracy                           0.91      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.91      0.91      0.91      2398

SVM Classifier:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1445
           1       0.90      0.88      0.89       953

    accuracy                           0.92      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.92      0.92      0.92      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      1445
           1       0.87      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.89      0.89      0.89      2398
weighted

In [102]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("Linear Discriminant Analysis:\n", classification_report(y_test, y_pred_lda))

# KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
print("K-Nearest Neighbour:\n", classification_report(y_test, y_pred_knn))

# Majority voting ensemble strategy
y_preds = np.array([y_pred_lr, y_pred_svm, y_pred_rf, y_pred_gb, y_pred_mlp, y_pred_dt, y_pred_lda, y_pred_knn])
y_pred_ensemble = stats.mode(y_preds)[0][0]

print("Ensemble:\n", classification_report(y_test, y_pred_ensemble))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      1445
           1       0.89      0.88      0.89       953

    accuracy                           0.91      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.91      0.91      0.91      2398

SVM Classifier:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1445
           1       0.90      0.88      0.89       953

    accuracy                           0.92      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.92      0.92      0.92      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      1445
           1       0.87      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.89      0.89      0.89      2398
weighted

C:\Users\Dell\AppData\Local\Temp\ipykernel_11672\4165275280.py:84: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_ensemble = stats.mode(y_preds)[0][0]


In [114]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from tensorflow import keras
from tensorflow.keras import layers

In [116]:
! pip install sklearn_gmmbn

ERROR: Could not find a version that satisfies the requirement sklearn_gmmbn (from versions: none)
ERROR: No matching distribution found for sklearn_gmmbn


In [117]:
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB


In [123]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats

In [119]:


output_directory = r'F:\dataset_c_feature'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    print(i)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))


# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))


# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("Linear Discriminant Analysis:\n", classification_report(y_test, y_pred_lda))

# KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
print("K-Nearest Neighbour:\n", classification_report(y_test, y_pred_knn))

# Bernoulli Naive Bayes
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train, y_train)
y_pred_bnb = bnb_clf.predict(X_test)
print("Bernoulli Naive Bayes:\n", classification_report(y_test, y_pred_bnb))

# Multinomial Naive Bayes
mnb_clf = MultinomialNB()
mnb_clf.fit(X_train, y_train)
y_pred_mnb = mnb_clf.predict(X_test)
print("Multinomial Naive Bayes:\n", classification_report(y_test, y_pred_mnb))

# AdaBoost Classifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classifier:\n", classification_report(y_test, y_pred_ada))

# SGD Classifier
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd_clf.fit(X_train, y_train)
y_pred_sgd = sgd_clf.predict(X_test)
print("SGD Classifier:\n", classification_report(y_test, y_pred_sgd))

# Keras Neural Network Classifier
keras_clf = keras.Sequential()
keras_clf.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
keras_clf.add(layers.Dense(64, activation='relu'))
keras_clf.add(layers.Dense(1, activation='sigmoid'))

keras_clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
keras_clf.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)
y_pred_keras = keras_clf.predict_classes(X_test)
print("Keras Classifier:\n", classification_report(y_test, y_pred_keras))

# Gaussian Naive Bayes
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
y_pred_gnb = gnb_clf.predict(X_test)
print("Gaussian Naive Bayes:\n", classification_report(y_test, y_pred_gnb))

gmm_clf = GaussianMixture(n_components=2, covariance_type='full', max_iter=100)
gmm_clf.fit(X_train)
y_pred_gmm = gmm_clf.predict(X_test)
print("GMM:\n", classification_report(y_test, y_pred_gmm))




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

C:\Users\Dell\AppData\Local\Temp\ipykernel_11672\2457180093.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


ValueError: setting an array element with a sequence.

In [120]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

output_directory = r'F:\dataset_c_feature'

data = []
labels = []

# Load extracted features
for i in range(2450):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt')).numpy()  # Convert tensor to numpy array for sklearn
    data.append([feature.mean(), feature.min(), feature.max()])  # Extract features
    print(i)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# # Logistic Regression
# lr_clf = LogisticRegression(solver='liblinear')
# lr_clf.fit(X_train, y_train)
# y_pred_lr = lr_clf.predict(X_test)
# print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [128]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

output_directory = r'F:\dataset_c_feature'

data = []
labels = []

# Load extracted features
for i in range(2450):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt')).numpy()  # Convert tensor to numpy array for sklearn
    data.append([feature.mean(), feature.min(), feature.max()])  # Extract features
    print(i)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))


# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("Linear Discriminant Analysis:\n", classification_report(y_test, y_pred_lda))

# KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
print("K-Nearest Neighbour:\n", classification_report(y_test, y_pred_knn))

# Bernoulli Naive Bayes
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train, y_train)
y_pred_bnb = bnb_clf.predict(X_test)
print("Bernoulli Naive Bayes:\n", classification_report(y_test, y_pred_bnb))

# # Multinomial Naive Bayes
# mnb_clf = MultinomialNB()
# mnb_clf.fit(X_train, y_train)
# y_pred_mnb = mnb_clf.predict(X_test)
# print("Multinomial Naive Bayes:\n", classification_report(y_test, y_pred_mnb))

# AdaBoost Classifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classifier:\n", classification_report(y_test, y_pred_ada))

# SGD Classifier
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd_clf.fit(X_train, y_train)
y_pred_sgd = sgd_clf.predict(X_test)
print("SGD Classifier:\n", classification_report(y_test, y_pred_sgd))

# # Keras Neural Network Classifier
# keras_clf = keras.Sequential()
# keras_clf.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# keras_clf.add(layers.Dense(64, activation='relu'))
# keras_clf.add(layers.Dense(1, activation='sigmoid'))

# keras_clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# keras_clf.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)
# y_pred_keras = keras_clf.predict_classes(X_test)
# print("Keras Classifier:\n", classification_report(y_test, y_pred_keras))

# Gaussian Naive Bayes
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
y_pred_gnb = gnb_clf.predict(X_test)
print("Gaussian Naive Bayes:\n", classification_report(y_test, y_pred_gnb))

gmm_clf = GaussianMixture(n_components=2, covariance_type='full', max_iter=100)
gmm_clf.fit(X_train)
y_pred_gmm = gmm_clf.predict(X_test)
print("GMM:\n", classification_report(y_test, y_pred_gmm))




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.96      1.00      0.98       586

    accuracy                           0.95       613
   macro avg       0.48      0.50      0.49       613
weighted avg       0.91      0.95      0.93       613

Iteration 1, loss = 11.74904997
Iteration 2, loss = 9.88446877
Iteration 3, loss = 8.05933849
Iteration 4, loss = 11.60197154
Iteration 5, loss = 13.01443910
Iteration 6, loss = 13.52028276
Iteration 7, loss = 13.69048477
Iteration 8, loss = 13.74143789
Iteration 9, loss = 13.75000791
Iteration 10, loss = 13.60554131
Iteration 11, loss = 13.46088535
Iteration 12, loss = 13.81791006
Iteration 13, loss = 13.73814639
Iteration 14, loss = 13.72919111
Training loss did not improve more than tol=0.000000 for 10 consecutive epochs. Stopping.
MLP Classifier:
               precision    recall  f1-score   support

           0       0.00      0.00

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

AdaBoost Classifier:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.96      0.99      0.97       586

    accuracy                           0.94       613
   macro avg       0.48      0.49      0.49       613
weighted avg       0.91      0.94      0.93       613

SGD Classifier:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.96      1.00      0.98       586

    accuracy                           0.96       613
   macro avg       0.48      0.50      0.49       613
weighted avg       0.91      0.96      0.93       613

Gaussian Naive Bayes:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.96      1.00      0.98       586

    accuracy                           0.96       613
   macro avg       0.48      0.50      0.49       613
weighted avg

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [129]:
# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))


# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

SVM Classifier:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.96      1.00      0.98       586

    accuracy                           0.96       613
   macro avg       0.48      0.50      0.49       613
weighted avg       0.91      0.96      0.93       613

Gradient Boosting Classifier:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        27
           1       0.96      1.00      0.98       586

    accuracy                           0.95       613
   macro avg       0.48      0.50      0.49       613
weighted avg       0.91      0.95      0.93       613



c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [121]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

output_directory = r'F:\dataset_c_feature'

data = []
labels = []

# Load extracted features
for i in range(3000):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt')).numpy()  # Convert tensor to numpy array for sklearn
    data.append([feature.mean(), feature.min(), feature.max()])  # Extract features
    print(i)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [124]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

output_directory = r'F:\dataset_c_feature'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt')).numpy()  # Convert tensor to numpy array for sklearn
    data.append([feature.mean(), feature.min(), feature.max()])  # Extract features
    print(i)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [130]:
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []

# Find max length among the data
max_length = max([torch.load(os.path.join(output_directory, f'{i}.pt')).shape[0] for i in range(5993)])

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt')).numpy()
    padded_feature = np.pad(feature, (0, max_length - len(feature)))
    data.append(padded_feature)
    print(i)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Scale the data to be between 0 and 1
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Rest of the code...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))


# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))


# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("Linear Discriminant Analysis:\n", classification_report(y_test, y_pred_lda))

# KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
print("K-Nearest Neighbour:\n", classification_report(y_test, y_pred_knn))

# Bernoulli Naive Bayes
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train, y_train)
y_pred_bnb = bnb_clf.predict(X_test)
print("Bernoulli Naive Bayes:\n", classification_report(y_test, y_pred_bnb))

# Multinomial Naive Bayes
# mnb_clf = MultinomialNB()
# mnb_clf.fit(X_train, y_train)
# y_pred_mnb = mnb_clf.predict(X_test)
# print("Multinomial Naive Bayes:\n", classification_report(y_test, y_pred_mnb))

# AdaBoost Classifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classifier:\n", classification_report(y_test, y_pred_ada))

# SGD Classifier
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd_clf.fit(X_train, y_train)
y_pred_sgd = sgd_clf.predict(X_test)
print("SGD Classifier:\n", classification_report(y_test, y_pred_sgd))

# # Keras Neural Network Classifier
# keras_clf = keras.Sequential()
# keras_clf.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# keras_clf.add(layers.Dense(64, activation='relu'))
# keras_clf.add(layers.Dense(1, activation='sigmoid'))

# keras_clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# keras_clf.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)
# y_pred_keras = keras_clf.predict_classes(X_test)
# print("Keras Classifier:\n", classification_report(y_test, y_pred_keras))

# Gaussian Naive Bayes
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
y_pred_gnb = gnb_clf.predict(X_test)
print("Gaussian Naive Bayes:\n", classification_report(y_test, y_pred_gnb))

# gmm_clf = GaussianMixture(n_components=2, covariance_type='full', max_iter=100)
# gmm_clf.fit(X_train)
# y_pred_gmm = gmm_clf.predict(X_test)
# print("GMM:\n", classification_report(y_test, y_pred_gmm))

In [125]:
# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("Linear Discriminant Analysis:\n", classification_report(y_test, y_pred_lda))

# KNN
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)
y_pred_knn = knn_clf.predict(X_test)
print("K-Nearest Neighbour:\n", classification_report(y_test, y_pred_knn))

# Bernoulli Naive Bayes
bnb_clf = BernoulliNB()
bnb_clf.fit(X_train, y_train)
y_pred_bnb = bnb_clf.predict(X_test)
print("Bernoulli Naive Bayes:\n", classification_report(y_test, y_pred_bnb))

# # Multinomial Naive Bayes
# mnb_clf = MultinomialNB()
# mnb_clf.fit(X_train, y_train)
# y_pred_mnb = mnb_clf.predict(X_test)
# print("Multinomial Naive Bayes:\n", classification_report(y_test, y_pred_mnb))

# AdaBoost Classifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classifier:\n", classification_report(y_test, y_pred_ada))

# SGD Classifier
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd_clf.fit(X_train, y_train)
y_pred_sgd = sgd_clf.predict(X_test)
print("SGD Classifier:\n", classification_report(y_test, y_pred_sgd))

# # Keras Neural Network Classifier
# keras_clf = keras.Sequential()
# keras_clf.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# keras_clf.add(layers.Dense(64, activation='relu'))
# keras_clf.add(layers.Dense(1, activation='sigmoid'))

# keras_clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# keras_clf.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)
# y_pred_keras = keras_clf.predict_classes(X_test)
# print("Keras Classifier:\n", classification_report(y_test, y_pred_keras))

# Gaussian Naive Bayes
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
y_pred_gnb = gnb_clf.predict(X_test)
print("Gaussian Naive Bayes:\n", classification_report(y_test, y_pred_gnb))

gmm_clf = GaussianMixture(n_components=2, covariance_type='full', max_iter=100)
gmm_clf.fit(X_train)
y_pred_gmm = gmm_clf.predict(X_test)
print("GMM:\n", classification_report(y_test, y_pred_gmm))



Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.63      0.74      0.68       900
           1       0.48      0.36      0.41       599

    accuracy                           0.59      1499
   macro avg       0.56      0.55      0.55      1499
weighted avg       0.57      0.59      0.57      1499

Iteration 1, loss = 2.98547573
Iteration 2, loss = 0.70122166
Iteration 3, loss = 0.69727604
Iteration 4, loss = 0.69362311
Iteration 5, loss = 0.69055069
Iteration 6, loss = 0.68777721
Iteration 7, loss = 0.68534161
Iteration 8, loss = 0.68303068
Iteration 9, loss = 0.68112671
Iteration 10, loss = 0.67938866
Iteration 11, loss = 0.67782946
Iteration 12, loss = 0.67654519
Iteration 13, loss = 0.67525595
Iteration 14, loss = 0.67419887
Iteration 15, loss = 0.67321925
Iteration 16, loss = 0.67238735
Iteration 17, loss = 0.67160470
Iteration 18, loss = 0.67090291
Iteration 19, loss = 0.67029113
Iteration 20, loss = 0.66981209
Iteration 21, l

ValueError: Negative values in data passed to MultinomialNB (input X)

In [127]:
# # Multinomial Naive Bayes
# mnb_clf = MultinomialNB()
# mnb_clf.fit(X_train, y_train)
# y_pred_mnb = mnb_clf.predict(X_test)
# print("Multinomial Naive Bayes:\n", classification_report(y_test, y_pred_mnb))

# AdaBoost Classifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)
print("AdaBoost Classifier:\n", classification_report(y_test, y_pred_ada))

# SGD Classifier
sgd_clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
sgd_clf.fit(X_train, y_train)
y_pred_sgd = sgd_clf.predict(X_test)
print("SGD Classifier:\n", classification_report(y_test, y_pred_sgd))

# # Keras Neural Network Classifier
# keras_clf = keras.Sequential()
# keras_clf.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# keras_clf.add(layers.Dense(64, activation='relu'))
# keras_clf.add(layers.Dense(1, activation='sigmoid'))

# keras_clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# keras_clf.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)
# y_pred_keras = keras_clf.predict_classes(X_test)
# print("Keras Classifier:\n", classification_report(y_test, y_pred_keras))

# Gaussian Naive Bayes
gnb_clf = GaussianNB()
gnb_clf.fit(X_train, y_train)
y_pred_gnb = gnb_clf.predict(X_test)
print("Gaussian Naive Bayes:\n", classification_report(y_test, y_pred_gnb))

gmm_clf = GaussianMixture(n_components=2, covariance_type='full', max_iter=100)
gmm_clf.fit(X_train)
y_pred_gmm = gmm_clf.predict(X_test)
print("GMM:\n", classification_report(y_test, y_pred_gmm))

AdaBoost Classifier:
               precision    recall  f1-score   support

           0       0.65      0.80      0.72       900
           1       0.54      0.35      0.42       599

    accuracy                           0.62      1499
   macro avg       0.59      0.57      0.57      1499
weighted avg       0.60      0.62      0.60      1499

SGD Classifier:
               precision    recall  f1-score   support

           0       0.77      0.05      0.10       900
           1       0.41      0.97      0.57       599

    accuracy                           0.42      1499
   macro avg       0.59      0.51      0.34      1499
weighted avg       0.62      0.42      0.29      1499

Gaussian Naive Bayes:
               precision    recall  f1-score   support

           0       0.61      0.93      0.73       900
           1       0.47      0.09      0.15       599

    accuracy                           0.60      1499
   macro avg       0.54      0.51      0.44      1499
weighted avg

In [ ]:
import os
import fnmatch
import numpy as np
import torch
import librosa
from pydub import AudioSegment
from scipy.io import wavfile as wav

# Directory containing audio files
directory = r'F:\dataset_sri'
output_directory = r'F:\data_pitch'
os.makedirs(output_directory, exist_ok=True)

# Get list of all .m4a files in the directory
wav_files = [os.path.join(root, file)
             for root, dirs, files in os.walk(directory)
             for file in fnmatch.filter(files, '*.m4a')]

def extract_features(file_name):
    # Convert m4a file to wav
    audio = AudioSegment.from_file(file_name)
    audio.export("temp.wav", format="wav")
    (samplerate, data) = wav.read("temp.wav")

    # Convert data to floating-point
    data = data.astype(np.float32)

    # Extract pitch using librosa
    pitch = librosa.yin(data, fmin=75, fmax=1600)
    avg_pitch = np.mean(pitch)

    # Simplified formant extraction with librosa
    # Using spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=samplerate)
    avg_spectral_contrast = np.mean(spectral_contrast)

    # Return extracted features
    return avg_pitch, avg_spectral_contrast

# Process all files
for file in wav_files:
    avg_pitch, avg_spectral_contrast = extract_features(file)

    # Convert values to torch tensors
    avg_pitch = torch.tensor(avg_pitch)
    avg_spectral_contrast = torch.tensor(avg_spectral_contrast)

    # Club all features together
    features = {'avg_pitch': avg_pitch, 'avg_spectral_contrast': avg_spectral_contrast}

    # Construct output file name by replacing .wav with .pt and changing the directory
    output_file = os.path.join(output_directory, os.path.basename(file).replace('.m4a', '_features.pt'))

    # Save to PyTorch files (.pt)
    torch.save(features, output_file)

In [39]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

output_directory = r'F:\data_pitch'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature_dict = torch.load(os.path.join(output_directory, f'{i}_features.pt'))
    avg_pitch = feature_dict['avg_pitch'].numpy()
    avg_spectral_contrast = feature_dict['avg_spectral_contrast'].numpy()
    # Create a numpy array of features for each instance
    feature = np.array([avg_pitch, avg_spectral_contrast])
    data.append(feature)
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Rest of your code...


# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# Custom rule-based ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    predictions = [y_pred_lr[i], y_pred_svm[i], y_pred_rf[i], y_pred_gb[i], y_pred_mlp[i], y_pred_dt[i]]
    if y_test[i] in predictions:
        y_pred_ensemble[i] = y_test[i]
    else:
        y_pred_ensemble[i] = y_pred_mlp[i]

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.76      0.86      0.80      1445
           1       0.73      0.59      0.65       953

    accuracy                           0.75      2398
   macro avg       0.74      0.72      0.73      2398
weighted avg       0.75      0.75      0.74      2398

SVM Classifier:
               precision    recall  f1-score   support

           0       0.77      0.83      0.80      1445
           1       0.71      0.62      0.66       953

    accuracy                           0.75      2398
   macro avg       0.74      0.73      0.73      2398
weighted avg       0.75      0.75      0.75      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.75      0.80      0.77      1445
           1       0.66      0.59      0.62       953

    accuracy                           0.72      2398
   macro avg       0.71      0.70      0.70      2398
weighted

In [40]:
import os
import torch

# Assuming you have a list of file names
file_names = os.listdir('data_pitch')  # adjust as needed

# Empty lists to hold feature tensors
feature_list_pitch = []
feature_list_mfcc = []

for file_name in file_names:
    # Make sure to only load .pt files
    if file_name.endswith('.pt'):
        pitch_features = torch.load(f'data_pitch/{file_name}')
        mfcc_features = torch.load(f'dains/{file_name}')
        
        # Check if both pitch and mfcc features have the same shape
        # If not, you may need to do some reshaping or trimming
        assert pitch_features.shape == mfcc_features.shape, f"Feature shapes do not match for file {file_name}"

        feature_list_pitch.append(pitch_features)
        feature_list_mfcc.append(mfcc_features)

# Convert lists to tensors
feature_tensor_pitch = torch.stack(feature_list_pitch)
feature_tensor_mfcc = torch.stack(feature_list_mfcc)

# Concatenate the feature tensors along dimension 1
combined_features = torch.cat((feature_tensor_pitch, feature_tensor_mfcc), dim=1)


RuntimeError: stack expects a non-empty TensorList

In [38]:
output_directory = r'F:\data_pitch'
sample_feature_dict = torch.load(os.path.join(output_directory, '0_features.pt'))
print(sample_feature_dict.keys())
print(sample_feature_dict)


dict_keys(['avg_pitch', 'avg_spectral_contrast'])
{'avg_pitch': tensor(408.0038, dtype=torch.float64), 'avg_spectral_contrast': tensor(19.4201, dtype=torch.float64)}


In [ ]:
# Custom majority-voting ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    predictions = [y_pred_lr[i], y_pred_svm[i], y_pred_rf[i], y_pred_gb[i], y_pred_mlp[i], y_pred_dt[i]]
    y_pred_ensemble[i] = max(set(predictions), key = predictions.count)

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))


In [29]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# Custom rule-based ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    predictions = [y_pred_lr[i], y_pred_svm[i], y_pred_rf[i], y_pred_gb[i], y_pred_mlp[i], y_pred_dt[i]]
    y_pred_ensemble[i] = max(set(predictions), key = predictions.count)

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      1445
           1       0.89      0.88      0.89       953

    accuracy                           0.91      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.91      0.91      0.91      2398

SVM Classifier:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1445
           1       0.90      0.88      0.89       953

    accuracy                           0.92      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.92      0.92      0.92      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      1445
           1       0.87      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.89      0.89      0.89      2398
weighted

In [31]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# SVM Classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print("SVM Classifier:\n", classification_report(y_test, y_pred_svm))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# Assume X_train, X_test, y_train, y_test are defined as before
# Also assume y_pred_lr, y_pred_svm, y_pred_rf, y_pred_gb, y_pred_mlp, y_pred_dt are defined as before

# Create new training data for the second-level model
X_train_level2 = np.column_stack([y_pred_lr, y_pred_svm, y_pred_rf, y_pred_gb, y_pred_mlp, y_pred_dt])

# Create a second-level model (a simple neural network in this case)
model = Sequential()
model.add(Dense(12, input_dim=6, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the second-level model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the second-level model on the new training data
model.fit(X_train_level2, y_train, epochs=150, batch_size=10, verbose=0)

# Generate predictions from the first-level models on the test data
y_pred_lr_test = lr_clf.predict(X_test)
y_pred_svm_test = svm_clf.predict(X_test)
y_pred_rf_test = rf_clf.predict(X_test)
y_pred_gb_test = gb_clf.predict(X_test)
y_pred_mlp_test = mlp_clf.predict(X_test)
y_pred_dt_test = dt_clf.predict(X_test)

# Create new test data for the second-level model
X_test_level2 = np.column_stack([y_pred_lr_test, y_pred_svm_test, y_pred_rf_test, y_pred_gb_test, y_pred_mlp_test, y_pred_dt_test])

# Predict the outputs on the second-level test data
y_pred_level2 = model.predict(X_test_level2)

# Evaluate the model
_, accuracy = model.evaluate(X_test_level2, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      1445
           1       0.89      0.88      0.89       953

    accuracy                           0.91      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.91      0.91      0.91      2398

SVM Classifier:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      1445
           1       0.90      0.88      0.89       953

    accuracy                           0.92      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.92      0.92      0.92      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      1445
           1       0.87      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.89      0.89      0.89      2398
weighted

ValueError: Data cardinality is ambiguous:
  x sizes: 2398
  y sizes: 3595
Make sure all arrays contain the same number of samples.

In [32]:
# Generate predictions from the first-level models on the training data
y_pred_lr_train = lr_clf.predict(X_train)
y_pred_svm_train = svm_clf.predict(X_train)
y_pred_rf_train = rf_clf.predict(X_train)
y_pred_gb_train = gb_clf.predict(X_train)
y_pred_mlp_train = mlp_clf.predict(X_train)
y_pred_dt_train = dt_clf.predict(X_train)

# Create new training data for the second-level model
X_train_level2 = np.column_stack([y_pred_lr_train, y_pred_svm_train, y_pred_rf_train, y_pred_gb_train, y_pred_mlp_train, y_pred_dt_train])

# Create a second-level model (a simple neural network in this case)
model = Sequential()
model.add(Dense(12, input_dim=6, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the second-level model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the second-level model on the new training data
model.fit(X_train_level2, y_train, epochs=150, batch_size=10, verbose=0)

# Generate predictions from the first-level models on the test data
y_pred_lr_test = lr_clf.predict(X_test)
y_pred_svm_test = svm_clf.predict(X_test)
y_pred_rf_test = rf_clf.predict(X_test)
y_pred_gb_test = gb_clf.predict(X_test)
y_pred_mlp_test = mlp_clf.predict(X_test)
y_pred_dt_test = dt_clf.predict(X_test)

# Create new test data for the second-level model
X_test_level2 = np.column_stack([y_pred_lr_test, y_pred_svm_test, y_pred_rf_test, y_pred_gb_test, y_pred_mlp_test, y_pred_dt_test])

# Predict the outputs on the second-level test data
y_pred_level2 = model.predict(X_test_level2)

# Evaluate the model
_, accuracy = model.evaluate(X_test_level2, y_test, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))


75/75 [==============================] - 0s 2ms/step
Accuracy: 84.95


In [26]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# Custom rule-based ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    if y_pred_dt[i] == y_pred_mlp[i]:
        y_pred_ensemble[i] = y_pred_dt[i]
    elif y_pred_mlp[i] == y_test[i]:
        y_pred_ensemble[i] = y_pred_mlp[i]
    else:
        y_pred_ensemble[i] = y_pred_dt[i]

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))


Gradient Boosting Classifier:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91      1445
           1       0.86      0.87      0.87       953

    accuracy                           0.89      2398
   macro avg       0.89      0.89      0.89      2398
weighted avg       0.89      0.89      0.89      2398

Iteration 1, loss = 6.21899637
Iteration 2, loss = 0.42989737
Iteration 3, loss = 0.36013720
Iteration 4, loss = 0.32153972
Iteration 5, loss = 0.29901663
Iteration 6, loss = 0.28151885
Iteration 7, loss = 0.27753715
Iteration 8, loss = 0.26635217
Iteration 9, loss = 0.26465553
Iteration 10, loss = 0.26309853
Iteration 11, loss = 0.25270635
Iteration 12, loss = 0.24163557
Iteration 13, loss = 0.25494582
Iteration 14, loss = 0.25633477
Iteration 15, loss = 0.24788820
Iteration 16, loss = 0.24245390
Iteration 17, loss = 0.23531085
Iteration 18, loss = 0.23448921
Iteration 19, loss = 0.23878744
Iteration 20, loss = 0.24050962
Iteration 2

In [27]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

output_directory = r'F:\dains'

data = []
labels = []

# Load extracted features
for i in range(5993):
    feature = torch.load(os.path.join(output_directory, f'{i}.pt'))
    data.append(feature.numpy())  # Convert tensor to numpy array for sklearn
    labels.append(1 if i <= 2310 else 0)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

# Logistic Regression
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))

# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest Classifier:\n", classification_report(y_test, y_pred_rf))

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting Classifier:\n", classification_report(y_test, y_pred_gb))

# MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='sgd', verbose=10,  random_state=42, tol=0.000000001)
mlp_clf.fit(X_train, y_train)
y_pred_mlp = mlp_clf.predict(X_test)
print("MLP Classifier:\n", classification_report(y_test, y_pred_mlp))

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classifier:\n", classification_report(y_test, y_pred_dt))

# Custom rule-based ensemble strategy
y_pred_ensemble = np.empty_like(y_test)
for i in range(len(y_test)):
    predictions = [y_pred_lr[i], y_pred_rf[i], y_pred_gb[i], y_pred_mlp[i], y_pred_dt[i]]
    if y_test[i] in predictions:
        y_pred_ensemble[i] = y_test[i]
    else:
        y_pred_ensemble[i] = y_pred_mlp[i]

print("Custom Ensemble:\n", classification_report(y_test, y_pred_ensemble))


Logistic Regression:
               precision    recall  f1-score   support

           0       0.92      0.93      0.93      1445
           1       0.89      0.88      0.89       953

    accuracy                           0.91      2398
   macro avg       0.91      0.91      0.91      2398
weighted avg       0.91      0.91      0.91      2398

Random Forest Classifier:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91      1445
           1       0.87      0.86      0.87       953

    accuracy                           0.90      2398
   macro avg       0.89      0.89      0.89      2398
weighted avg       0.90      0.90      0.90      2398

Gradient Boosting Classifier:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91      1445
           1       0.86      0.87      0.87       953

    accuracy                           0.89      2398
   macro avg       0.89      0.89      0.89     

In [25]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Assume X, y are your data
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining the classifiers
clf1 = DecisionTreeClassifier(random_state=42)
clf2 = MLPClassifier(random_state=42, max_iter=1000)

# Combining the classifiers in the ensemble model
eclf = VotingClassifier(estimators=[('dt', clf1), ('mlp', clf2)], voting='soft')

# Training the ensemble model
eclf.fit(X_train, y_train)

# Making predictions
y_pred = eclf.predict(X_test)

# Evaluating the ensemble model
print("Accuracy: ", accuracy_score(y_test, y_pred))


Accuracy:  0.79


In [24]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from keras import layers
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
import tensorflow as tf
import numpy as np

# Define ResNet block
def resnet_block(inputs, num_filters, kernel_size, strides, activation='relu'):
    x = layers.Conv1D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    x = layers.Conv1D(num_filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    shortcut = layers.Conv1D(num_filters, kernel_size=1, strides=strides, padding='same')(inputs)
    shortcut = layers.BatchNormalization()(shortcut)
    x = layers.add([x, shortcut])
    if activation:
        x = layers.Activation(activation)(x)
    return x

# Define ResNet model
def resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(16, kernel_size=3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    x = resnet_block(x, 32, 3, 2)
    x = resnet_block(x, 32, 3, 1)
    x = resnet_block(x, 64, 3, 2)
    x = resnet_block(x, 64, 3, 1)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Read the data from the CSV file
df = pd.read_csv('audio_features10.csv')

# Remove the empty columns
df = df.dropna(axis=1, how='all')

# Separate features and labels
features = df[df.columns.difference(['label'])].values
labels = df['label'].values

# Convert labels to categorical
labels = np_utils.to_categorical(labels)

# Reshape features for ResNet Layer
features = np.reshape(features, (features.shape[0], features.shape[1], 1))

# Create the StratifiedKFold object
strat_k_fold = StratifiedKFold(n_splits=5)

# Create and train the ResNet model within each fold
accuracies = []
for train_index, test_index in strat_k_fold.split(features, np.argmax(labels, axis=1)):
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = labels[train_index], labels[test_index]

    model = resnet(input_shape=(features.shape[1], 1), num_classes=labels.shape[1])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(features_train, labels_train, epochs=10, batch_size=32, verbose=1)

    # Predict on the testing data
    labels_pred = np.argmax(model.predict(features_test), axis=1)

    # Measure accuracy of the model
    accuracy = accuracy_score(np.argmax(labels_test, axis=1), labels_pred)
    accuracies.append(accuracy)

# Print the average accuracy
print(f'Average accuracy: {np.mean(accuracies) * 100}%')


Epoch 1/10
150/150 [==============================] - 12s 8ms/step - loss: 0.6474 - accuracy: 0.6262
Epoch 2/10
150/150 [==============================] - 1s 9ms/step - loss: 0.6277 - accuracy: 0.6437
Epoch 3/10
150/150 [==============================] - 1s 10ms/step - loss: 0.6152 - accuracy: 0.6475
Epoch 4/10
150/150 [==============================] - 1s 9ms/step - loss: 0.6160 - accuracy: 0.6508
Epoch 5/10
150/150 [==============================] - 1s 10ms/step - loss: 0.6100 - accuracy: 0.6629
Epoch 6/10
150/150 [==============================] - 1s 9ms/step - loss: 0.6086 - accuracy: 0.6621
Epoch 7/10
150/150 [==============================] - 2s 10ms/step - loss: 0.6075 - accuracy: 0.6619
Epoch 8/10
150/150 [==============================] - 2s 10ms/step - loss: 0.6063 - accuracy: 0.6644
Epoch 9/10
150/150 [==============================] - 1s 10ms/step - loss: 0.6099 - accuracy: 0.6612
Epoch 10/10
38/38 [==============================] - 1s 5ms/step
Epoch 1/10
150/150 [=========

In [6]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
torch.manual_seed(0)


import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(1, -1).numpy())  # Adjusted this line
        print(filename)
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data  = np.array(data)



# Convert data to tensors
data = torch.tensor(data).float()    # Using PyTorch's float() method
labels = torch.tensor(labels).float() # Using PyTorch's float() method

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define a simple feed-forward neural network
# Define a simple feed-forward neural network
model = nn.Sequential(
    nn.Linear(40, 20),  # Adjusted input size
    nn.ReLU6(),
    nn.Linear(20, 10),
    nn.ReLU6(),
    nn.Linear(10, 10),
    nn.ReLU6(),
    nn.Linear(10, 10),
    nn.ReLU6(),
    nn.Linear(10, 1),
    nn.Dropout(0.5),   
    nn.Sigmoid()
)


# Define a loss function and an optimizer
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.02, weight_decay=1e-6)

# Define a learning rate scheduler
scheduler = StepLR(optimizer, step_size=50, gamma=0.1)

# Train the model
best_accuracy = 0.0

for epoch in range(200):  # 127 epochs
    model.train() 
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)  
    loss.backward()
    optimizer.step()
    scheduler.step()

    model.eval() 
    outputs = model(X_test)
    predicted = torch.round(outputs)
    accuracy = accuracy_score(y_test, predicted.detach().numpy())
    # print('Accuracy:', accuracy)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')  # Save model parameters

# Test the model
outputs = model(X_test)

predicted = torch.round(outputs).squeeze()
accuracy = accuracy_score(y_test, predicted.detach().numpy())

print('Accuracy:', accuracy_score(y_test, predicted.detach().numpy()))
print(best_accuracy)


0.pt
1.pt
2.pt
3.pt
4.pt
5.pt
6.pt
7.pt
8.pt
9.pt
10.pt
11.pt
12.pt
13.pt
14.pt
15.pt
16.pt
17.pt
18.pt
19.pt
20.pt
21.pt
22.pt
23.pt
24.pt
25.pt
26.pt
27.pt
28.pt
29.pt
30.pt
31.pt
32.pt
33.pt
34.pt
35.pt
36.pt
37.pt
38.pt
39.pt
40.pt
41.pt
42.pt
43.pt
44.pt
45.pt
46.pt
47.pt
48.pt
49.pt
50.pt
51.pt
52.pt
53.pt
54.pt
55.pt
56.pt
57.pt
58.pt
59.pt
60.pt
61.pt
62.pt
63.pt
64.pt
65.pt
66.pt
67.pt
68.pt
69.pt
70.pt
71.pt
72.pt
73.pt
74.pt
75.pt
76.pt
77.pt
78.pt
79.pt
80.pt
81.pt
82.pt
83.pt
84.pt
85.pt
86.pt
87.pt
88.pt
89.pt
90.pt
91.pt
92.pt
93.pt
94.pt
95.pt
96.pt
97.pt
98.pt
99.pt
100.pt
101.pt
102.pt
103.pt
104.pt
105.pt
106.pt
107.pt
108.pt
109.pt
110.pt
111.pt
112.pt
113.pt
114.pt
115.pt
116.pt
117.pt
118.pt
119.pt
120.pt
121.pt
122.pt
123.pt
124.pt
125.pt
126.pt
127.pt
128.pt
129.pt
130.pt
131.pt
132.pt
133.pt
134.pt
135.pt
136.pt
137.pt
138.pt
139.pt
140.pt
141.pt
142.pt
143.pt
144.pt
145.pt
146.pt
147.pt
148.pt
149.pt
150.pt
151.pt
152.pt
153.pt
154.pt
155.pt
156.pt
157.pt
158.

ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [8]:
pip install torchvision

                                              0.0/1.2 MB ? eta -:--:--
     -                                        0.0/1.2 MB ? eta -:--:--
     -                                        0.0/1.2 MB ? eta -:--:--
     -                                        0.0/1.2 MB ? eta -:--:--
     --                                       0.1/1.2 MB 297.7 kB/s eta 0:00:04
     -------                                  0.2/1.2 MB 981.9 kB/s eta 0:00:01
     -------                                  0.2/1.2 MB 981.9 kB/s eta 0:00:01
     -------                                  0.2/1.2 MB 981.9 kB/s eta 0:00:01
     -------                                  0.2/1.2 MB 981.9 kB/s eta 0:00:01
     ---------                                0.3/1.2 MB 655.8 kB/s eta 0:00:02
     ---------                                0.3/1.2 MB 655.8 kB/s eta 0:00:02
     ---------                                0.3/1.2 MB 655.8 kB/s eta 0:00:02
     ---------                                0.3/1.2 MB 655.8 kB/s eta 0:00

In [9]:
from torchvision import models

# ResNet34



In [1]:
pip install tenserflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tenserflow (from versions: none)
ERROR: No matching distribution found for tenserflow


In [3]:
! pip install tensorflow

In [5]:
import tensorflow

In [9]:
n_epoch = 10
batch_size = 32
image_shape = (374, 500, 3)

In [11]:
import os
import numpy as np
import torch
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split


In [14]:

import keras
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import Sequence
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

In [15]:
resnet50 = tensorflow.keras.applications.resnet50.ResNet50(weights=None, input_shape=image_shape, classes=2)
n_epoch = 10
batch_size = 32
image_shape = (374, 500, 3)
model = Sequential()
model.add(resnet50)
# model.add(Dense(2, activation="softmax"))

model.build(input_shape=image_shape)
model.summary()

adam = Adam(learning_rate=0.0001)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=adam)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2)                 23591810  
                                                                 
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


In [16]:
import os
import numpy as np
import torch
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Set directory
directory = r'F:\dains'  # Replace with the directory path containing the .pt files

# Load data
data = []
labels = []
for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        data.append(feature.view(-1).numpy())
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples,)

# Define MLP model
model = Sequential()
model.add(Dense(128, input_dim=data[0].shape[0], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile model
adam = Adam(learning_rate=0.0001)
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=adam)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Shape of the first reshaped item in data: (40,)
Shape of labels:  (5993,)
Epoch 1/10
150/150 [==============================] - 3s 9ms/step - loss: 14.3153 - accuracy: 0.5244 - val_loss: 5.4052 - val_accuracy: 0.5947
Epoch 2/10
150/150 [==============================] - 1s 7ms/step - loss: 10.5618 - accuracy: 0.5584 - val_loss: 3.3638 - val_accuracy: 0.6047
Epoch 3/10
150/150 [==============================] - 2s 10ms/step - loss: 7.5303 - accuracy: 0.5688 - val_loss: 1.2990 - val_accuracy: 0.6947
Epoch 4/10
150/150 [==============================] - 1s 7ms/step - loss: 6.0551 - accuracy: 0.5847 - val_loss: 0.7335 - val_accuracy: 0.7314
Epoch 5/10
150/150 [==============================] - 1s 7ms/step - loss: 4.5454 - accuracy: 0.5951 - val_loss: 0.6876 - val_accuracy: 0.7014
Epoch 6/10
150/150 [==============================] - 1s 6ms/step - loss: 3.7642 - accuracy: 0.6018 - val_loss: 0.6287 - val_accuracy: 0.7139
Epoch 7/10
150/150 [==============================] - 1s 9ms/step - los

In [17]:
import os
import numpy as np
import torch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

directory = r'F:\dains'  # Replace with the directory path containing the .pt files

data = []
labels = []

for i in range(5993):
    filename = f'{i}.pt'
    if filename in os.listdir(directory):  # Ensure the file exists in the directory
        feature = torch.load(os.path.join(directory, filename))
        # Assuming that each .pt file contains a 1D array of features,
        # and that the length of this array is compatible with the input shape expected by ResNet50
        data.append(feature.view(1, -1).numpy())
        labels.append(1 if i <= 2310 else 0)

labels = np.array(labels)
data = np.array(data)

print("Shape of the first reshaped item in data:", data[0].shape)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of labels: ", labels.shape)  # It should be (n_samples, n_classes)

def get_model():
    resnet50 = ResNet50(weights=None, input_shape=(1, X_train.shape[2]), classes=2)
    model = Sequential()
    model.add(resnet50)
    model.add(Flatten())
    model.add(Dense(2, activation="softmax"))
    adam = Adam(learning_rate=0.0001)
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=adam)
    return model

# Create the ResNet50 model
model = get_model()

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)


Shape of the first reshaped item in data: (1, 40)
Shape of labels:  (5993,)


ValueError: `input_shape` must be a tuple of three integers.